### Table1

In [91]:
# Format as currency
import locale

def format_currency(value):
    if pd.isna(value):
        return
    return locale.currency(value, grouping=True).split('.')[0]

import tabula
# Extract tables from the PDF
# Function to set locale safely
def set_locale(locale_name):
    try:
        locale.setlocale(locale.LC_ALL, locale_name)
    except locale.Error:
        print(f"Locale {locale_name} not supported on this system. Using default locale.")
        locale.setlocale(locale.LC_ALL, '')

# Set the locale (adjust as necessary for your system)
set_locale('en_US.UTF-8')  # Common locale setting for Unix-like systems
locale._override_localeconv = {'n_sign_posn':1}
import pandas as pd
# Path to the PDF file
pdf_path = 'Data.pdf'
table = tabula.read_pdf(pdf_path, pages=2, multiple_tables=False)

Aug 03, 2024 6:15:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [92]:

import tabula

# Path to the PDF file
pdf_path = 'Data.pdf'

# Extract tables from the PDF (specify the page number)
tables = tabula.read_pdf(pdf_path, pages=2, multiple_tables=False)

# Convert the first table in the list to a DataFrame
df = pd.DataFrame(tables[0])


Aug 03, 2024 6:15:25 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



In [93]:
df

,April 30 January 31,April 30,April 30.1,April 30.2
0,2023(1)(Unaudited) ($ millions) 2024(1) 2024(1),2023(1),2024(1),NaN
1,Operating results,NaN,NaN,NaN
2,"Net interest income $ 4,694 $ 4,773","$ 4,460","$ 9,467","$ 9,023"
3,"Non-interest income 3,653 3,660","3,453","7,313","6,852"
4,"Total revenue $ 8,347 $ 8,433","$ 7,913","$ 16,780","$ 15,875"
5,"Provision for credit losses 1,007 962",709,"1,969","1,347"
6,"Non-interest expenses 4,711 4,739","4,574","9,450","9,035"
7,Income tax expense 537 533,484,"1,070","1,589"
8,"Net income $ 2,092 $ 2,199","$ 2,146","$ 4,291","$ 3,904"
9,Net income attributable to non-controlling int...,24,51,61


In [94]:
# Drop the first row (index 0)
df.drop(index=0, inplace=True)

# Reset the index to ensure it is continuous
df.reset_index(drop=True, inplace=True)

# Print the DataFrame to verify
print(df.head())

                     April 30 January 31 April 30 April 30.1 April 30.2
0                      Operating results      NaN        NaN        NaN
1    Net interest income $ 4,694 $ 4,773  $ 4,460    $ 9,467    $ 9,023
2        Non-interest income 3,653 3,660    3,453      7,313      6,852
3          Total revenue $ 8,347 $ 8,433  $ 7,913   $ 16,780   $ 15,875
4  Provision for credit losses 1,007 962      709      1,969      1,347


In [95]:
# Function to add dollar sign before numbers
def add_dollar_sign(text):
    # Function to add dollar sign to numbers
    def replace_with_dollar(match):
        return f'${match.group(0)}'

    # Use regular expression to find numbers
    return re.sub(r'\b\d+(?:,\d{3})*(?:\.\d+)?\b', replace_with_dollar, text)

# Apply the function
df['April 30 January 31'] = df['April 30 January 31'].apply(add_dollar_sign)

# Print the DataFrame to verify
print(df)

                                  April 30 January 31 April 30 April 30.1  \
0                                   Operating results      NaN        NaN   
1               Net interest income $ $4,694 $ $4,773  $ 4,460    $ 9,467   
2                   Non-interest income $3,653 $3,660    3,453      7,313   
3                     Total revenue $ $8,347 $ $8,433  $ 7,913   $ 16,780   
4             Provision for credit losses $1,007 $962      709      1,969   
5                 Non-interest expenses $4,711 $4,739    4,574      9,450   
6                        Income tax expense $537 $533      484      1,070   
7                        Net income $ $2,092 $ $2,199  $ 2,146    $ 4,291   
8   Net income attributable to non-controlling int...       24         51   
9   Net income attributable to equity holders of t...  $ 2,122    $ 4,240   
10  Preferred shareholders and other equity instru...      104        231   
11              Common shareholders $ $1,943 $ $2,066  $ 2,018    $ 4,009   

In [96]:
def split_dollars(row):
    if '$' in row:
        parts = row.split('$')
        # Remove empty strings and strip whitespaces
        return [parts[0].strip()] + [part.strip() for part in parts[1:]]
    else:
        return [row] + [''] * 3

# Apply the function and create new columns
df_split = df['April 30 January 31'].apply(split_dollars).apply(pd.Series)
df_split.columns = ['(Unaudited) ($ millions) ', 'Column1', 'Column2','Column3','Column5']

# Concatenate the new DataFrame with the original DataFrame (excluding the old column)
df1 = pd.concat([df_split, df.drop(columns='April 30 January 31')], axis=1)

# Print the updated DataFrame
print(df1)

                            (Unaudited) ($ millions)  Column1 Column2 Column3  \
0                                   Operating results                           
1                                 Net interest income           4,694           
2                                 Non-interest income   3,653   3,660     NaN   
3                                       Total revenue           8,347           
4                         Provision for credit losses   1,007     962     NaN   
5                               Non-interest expenses   4,711   4,739     NaN   
6                                  Income tax expense     537     533     NaN   
7                                          Net income           2,092           
8   Net income attributable to non-controlling int...      26      25     NaN   
9   Net income attributable to equity holders of t...           2,066           
10  Preferred shareholders and other equity instru...     123     108     NaN   
11                          

In [97]:

# Replace NaN values with empty strings
df1[['Column1', 'Column2', 'Column3', 'Column5']] = df1[['Column1', 'Column2', 'Column3', 'Column5']].fillna('')

# Convert columns to strings and merge them
df1['merged'] = df1['Column1'].astype(str) + df1['Column2'].astype(str) + df1['Column3'].astype(str) + df1['Column5'].astype(str)

# Split the merged column into two new columns
df1[['April 30 2024(For the three months ended)', 'January 31 2024(For the three months ended)']] = df1['merged'].apply(lambda x: pd.Series([x[:len(x)//2], x[len(x)//2:]]))

# Drop the intermediate merged column and original columns
df1 = df1.drop(columns=['merged', 'Column1', 'Column2', 'Column3', 'Column5'])

# Reorder the columns to place the new columns at indices 1 and 2
columns = list(df1.columns)
columns.insert(1, columns.pop(columns.index('April 30 2024(For the three months ended)')))
columns.insert(2, columns.pop(columns.index('January 31 2024(For the three months ended)')))
df1 = df1[columns]

df1


,(Unaudited) ($ millions),April 30 2024(For the three months ended),January 31 2024(For the three months ended),April 30,April 30.1,April 30.2
0,Operating results,,,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","$ 4,460","$ 9,467","$ 9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","$ 7,913","$ 16,780","$ 15,875"
4,Provision for credit losses,"1,00",7962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income tax expense,537,533,484,"1,070","1,589"
7,Net income,"2,092","2,199","$ 2,146","$ 4,291","$ 3,904"
8,Net income attributable to non-controlling int...,26,25,24,51,61
9,Net income attributable to equity holders of t...,"2,066","2,174","$ 2,122","$ 4,240","$ 3,843"


In [98]:
new_column_names = {
    'April 30': 'April 30 2023 (For the three months ended)',
    'April 30.1': 'April 30 2024 (For the six months ended)',
    'April 30.2': 'April 30 2023 (For the six months ended)'
}

# Rename the columns
df1 = df1.rename(columns=new_column_names)
df1

,(Unaudited) ($ millions),April 30 2024(For the three months ended),January 31 2024(For the three months ended),April 30 2023 (For the three months ended),April 30 2024 (For the six months ended),April 30 2023 (For the six months ended)
0,Operating results,,,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","$ 4,460","$ 9,467","$ 9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","$ 7,913","$ 16,780","$ 15,875"
4,Provision for credit losses,"1,00",7962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income tax expense,537,533,484,"1,070","1,589"
7,Net income,"2,092","2,199","$ 2,146","$ 4,291","$ 3,904"
8,Net income attributable to non-controlling int...,26,25,24,51,61
9,Net income attributable to equity holders of t...,"2,066","2,174","$ 2,122","$ 4,240","$ 3,843"


#### Add dollar symbol

In [99]:
import re

def add_dollar_sign(cell):
    if isinstance(cell, str):
        # Add dollar sign only to numbers, not to already formatted amounts or text
        return re.sub(r'(?<!\$)(?<!\d,)(\b\d+(?:,\d{3})*)', r'$\1', cell)
    return cell

# Assuming the first column is at index 0
first_column_name = df1.columns[0]

# Apply the function only to the first column
df1 = df1.copy()  # Create a copy to avoid modifying the original DataFrame
df1[first_column_name] = df1[first_column_name].apply(add_dollar_sign)

# Display the DataFrame with updated first column
print(df1)


                            (Unaudited) ($ millions)   \
0                                   Operating results   
1                                 Net interest income   
2                                 Non-interest income   
3                                       Total revenue   
4                         Provision for credit losses   
5                               Non-interest expenses   
6                                  Income tax expense   
7                                          Net income   
8   Net income attributable to non-controlling int...   
9   Net income attributable to equity holders of t...   
10  Preferred shareholders and other equity instru...   
11                                Common shareholders   
12             Earnings per common share (in dollars)   
13                                              Basic   
14                                            Diluted   

   April 30 2024(For the three months ended)  \
0                                      

In [100]:
df1

,(Unaudited) ($ millions),April 30 2024(For the three months ended),January 31 2024(For the three months ended),April 30 2023 (For the three months ended),April 30 2024 (For the six months ended),April 30 2023 (For the six months ended)
0,Operating results,,,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","$ 4,460","$ 9,467","$ 9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","$ 7,913","$ 16,780","$ 15,875"
4,Provision for credit losses,"1,00",7962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income tax expense,537,533,484,"1,070","1,589"
7,Net income,"2,092","2,199","$ 2,146","$ 4,291","$ 3,904"
8,Net income attributable to non-controlling int...,26,25,24,51,61
9,Net income attributable to equity holders of t...,"2,066","2,174","$ 2,122","$ 4,240","$ 3,843"


In [101]:
df1.drop(index=0, inplace=True)
df1.drop(index=12, inplace=True)
df1.reset_index(drop=True, inplace=True)
df1

,(Unaudited) ($ millions),April 30 2024(For the three months ended),January 31 2024(For the three months ended),April 30 2023 (For the three months ended),April 30 2024 (For the six months ended),April 30 2023 (For the six months ended)
0,Net interest income,"4,694","4,773","$ 4,460","$ 9,467","$ 9,023"
1,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
2,Total revenue,"8,347","8,433","$ 7,913","$ 16,780","$ 15,875"
3,Provision for credit losses,"1,00",7962,709,"1,969","1,347"
4,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
5,Income tax expense,537,533,484,"1,070","1,589"
6,Net income,"2,092","2,199","$ 2,146","$ 4,291","$ 3,904"
7,Net income attributable to non-controlling int...,26,25,24,51,61
8,Net income attributable to equity holders of t...,"2,066","2,174","$ 2,122","$ 4,240","$ 3,843"
9,Preferred shareholders and other equity instru...,123,108,104,231,205


#### Cleaned Table 1

In [129]:
df1

,(Unaudited) ($ millions),April 30 2024(For the three months ended),January 31 2024(For the three months ended),April 30 2023 (For the three months ended),April 30 2024 (For the six months ended),April 30 2023 (For the six months ended)
0,Net interest income,"4,694","4,773","$ 4,460","$ 9,467","$ 9,023"
1,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
2,Total revenue,"8,347","8,433","$ 7,913","$ 16,780","$ 15,875"
3,Provision for credit losses,"1,00",7962,709,"1,969","1,347"
4,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
5,Income tax expense,537,533,484,"1,070","1,589"
6,Net income,"2,092","2,199","$ 2,146","$ 4,291","$ 3,904"
7,Net income attributable to non-controlling int...,26,25,24,51,61
8,Net income attributable to equity holders of t...,"2,066","2,174","$ 2,122","$ 4,240","$ 3,843"
9,Preferred shareholders and other equity instru...,123,108,104,231,205


### Table 2

In [102]:
# Extract tables from the PDF (specify the page number)
tables = tabula.read_pdf(pdf_path, pages=7, multiple_tables=True)

# Convert the first table in the list to a DataFrame
df2 = pd.DataFrame(tables[0])
df2

Aug 03, 2024 6:22:10 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Unnamed: 0,Unnamed: 1,April 30,Unnamed: 2,January 31,Unnamed: 3,April 30.1,Unnamed: 4,April 30.2,Unnamed: 5,April 30.3
0,($ millions),NaN,2024(1),NaN,2024(1),NaN,2023(1),NaN,2024(1),NaN,2023(1)
1,Reported Results,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net interest income,$,"4,694",$,"4,773",$,"4,460",$,"9,467",$,"9,023"
3,Non-interest income,NaN,"3,653",NaN,"3,660",NaN,"3,453",NaN,"7,313",NaN,"6,852"
4,Total revenue,NaN,"8,347",NaN,"8,433",NaN,"7,913",NaN,"16,780",NaN,"15,875"
5,Provision for credit losses,NaN,"1,007",NaN,962,NaN,709,NaN,"1,969",NaN,"1,347"
6,Non-interest expenses,NaN,"4,711",NaN,"4,739",NaN,"4,574",NaN,"9,450",NaN,"9,035"
7,Income before taxes,NaN,"2,629",NaN,"2,732",NaN,"2,630",NaN,"5,361",NaN,"5,493"
8,Income tax expense,NaN,537,NaN,533,NaN,484,NaN,"1,070",NaN,"1,589"
9,Net income,$,"2,092",$,"2,199",$,"2,146",$,"4,291",$,"3,904"


In [103]:
df2.drop(columns=['Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'], inplace=True)
df2

,Unnamed: 0,April 30,January 31,April 30.1,April 30.2,April 30.3
0,($ millions),2024(1),2024(1),2023(1),2024(1),2023(1)
1,Reported Results,NaN,NaN,NaN,NaN,NaN
2,Net interest income,"4,694","4,773","4,460","9,467","9,023"
3,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
4,Total revenue,"8,347","8,433","7,913","16,780","15,875"
5,Provision for credit losses,"1,007",962,709,"1,969","1,347"
6,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
7,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
8,Income tax expense,537,533,484,"1,070","1,589"
9,Net income,"2,092","2,199","2,146","4,291","3,904"


In [104]:
# Extract the first row to create a new header
new_header = df2.columns + ' ' + df2.iloc[0]
# Set the new header
df2.columns = new_header

# Remove the first row from the DataFrame
df2 = df2[1:]

# Reset the index
df2.reset_index(drop=True, inplace=True)

# Display modified DataFrame
print("\nModified DataFrame with merged header:")
df2


Modified DataFrame with merged header:


,Unnamed: 0 ($ millions),April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Reported Results,NaN,NaN,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","4,460","9,467","9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","7,913","16,780","15,875"
4,Provision for credit losses,"1,007",962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
7,Income tax expense,537,533,484,"1,070","1,589"
8,Net income,"2,092","2,199","2,146","4,291","3,904"
9,Net income attributable to non-controlling int...,26,25,24,51,61


In [105]:

# Drop the original row 12
df2.drop(index=11, inplace=True)
df2.drop(index=15, inplace=True)
df2.drop(index=18, inplace=True)
df2.drop(index=22, inplace=True)
df2.drop(index=28, inplace=True)
df2.drop(index=41, inplace=True)
df2.drop(index=46, inplace=True)


# Reset the index
df2.reset_index(drop=True, inplace=True)
df2

<ipython-input-105-b00e83af7491>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=11, inplace=True)
<ipython-input-105-b00e83af7491>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=15, inplace=True)
<ipython-input-105-b00e83af7491>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=18, inplace=True)
<ipython-input-105-b00e83af7491>:5: SettingWithCopyWarning: 
A value is trying to be set on a 

,Unnamed: 0 ($ millions),April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Reported Results,NaN,NaN,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","4,460","9,467","9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","7,913","16,780","15,875"
4,Provision for credit losses,"1,007",962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
7,Income tax expense,537,533,484,"1,070","1,589"
8,Net income,"2,092","2,199","2,146","4,291","3,904"
9,Net income attributable to non-controlling int...,26,25,24,51,61


In [106]:
# Rename row 11 (index 10) value

df2.iloc[10, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Net income attributable to preferred shareholders and other equity instrument holders'
df2.iloc[14, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Weighted average number of diluted common shares outstanding (millions)'
df2.iloc[16, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Impact of adjusting items on income tax expense Canada recovery dividend Amortization of acquisition-related intangible assets'
df2.iloc[19, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Impact of adjusting items on income tax expense Canada recovery dividend Amortization of acquisition-related intangible assets'
df2.iloc[24, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Total impact of adjusting items on net income attributable to equity holders and common shareholders'
df2.iloc[36, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Net income attributable to preferred shareholders and other equity instrument holders'
df2.iloc[40, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Weighted average number of diluted common shares outstanding (millions)'
df2

<ipython-input-106-bd636b204f66>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.iloc[10, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Net income attributable to preferred shareholders and other equity instrument holders'
<ipython-input-106-bd636b204f66>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.iloc[14, df2.columns.get_loc('Unnamed: 0 ($ millions)')] = 'Weighted average number of diluted common shares outstanding (millions)'
<ipython-input-106-bd636b204f66>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

,Unnamed: 0 ($ millions),April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Reported Results,NaN,NaN,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","4,460","9,467","9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","7,913","16,780","15,875"
4,Provision for credit losses,"1,007",962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
7,Income tax expense,537,533,484,"1,070","1,589"
8,Net income,"2,092","2,199","2,146","4,291","3,904"
9,Net income attributable to non-controlling int...,26,25,24,51,61


In [126]:
df2.rename(columns={'Unnamed: 0 ($ millions)': '$ millions'}, inplace=True)
df2

<ipython-input-126-f123bb6bafe9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'Unnamed: 0 ($ millions)': '$ millions'}, inplace=True)


,$ millions,April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Reported Results,NaN,NaN,NaN,NaN,NaN
1,Net interest income,"4,694","4,773","4,460","9,467","9,023"
2,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
3,Total revenue,"8,347","8,433","7,913","16,780","15,875"
4,Provision for credit losses,"1,007",962,709,"1,969","1,347"
5,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
6,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
7,Income tax expense,537,533,484,"1,070","1,589"
8,Net income,"2,092","2,199","2,146","4,291","3,904"
9,Net income attributable to non-controlling int...,26,25,24,51,61


In [127]:
df2.drop(index=0, inplace=True)
df2.drop(index=15, inplace=True)
df2.drop(index=25, inplace=True)
df2.reset_index(drop=True, inplace=True)
df2

<ipython-input-127-eacfb50e2ad8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=0, inplace=True)
<ipython-input-127-eacfb50e2ad8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=15, inplace=True)
<ipython-input-127-eacfb50e2ad8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(index=25, inplace=True)


,$ millions,April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Net interest income,"4,694","4,773","4,460","9,467","9,023"
1,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
2,Total revenue,"8,347","8,433","7,913","16,780","15,875"
3,Provision for credit losses,"1,007",962,709,"1,969","1,347"
4,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
5,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
6,Income tax expense,537,533,484,"1,070","1,589"
7,Net income,"2,092","2,199","2,146","4,291","3,904"
8,Net income attributable to non-controlling int...,26,25,24,51,61
9,Net income attributable to preferred sharehold...,"2,066","2,174","2,122","4,240","3,843"


#### Cleaned Table 2

In [128]:
df2

,$ millions,April 30 2024(1),January 31 2024(1),April 30.1 2023(1),April 30.2 2024(1),April 30.3 2023(1)
0,Net interest income,"4,694","4,773","4,460","9,467","9,023"
1,Non-interest income,"3,653","3,660","3,453","7,313","6,852"
2,Total revenue,"8,347","8,433","7,913","16,780","15,875"
3,Provision for credit losses,"1,007",962,709,"1,969","1,347"
4,Non-interest expenses,"4,711","4,739","4,574","9,450","9,035"
5,Income before taxes,"2,629","2,732","2,630","5,361","5,493"
6,Income tax expense,537,533,484,"1,070","1,589"
7,Net income,"2,092","2,199","2,146","4,291","3,904"
8,Net income attributable to non-controlling int...,26,25,24,51,61
9,Net income attributable to preferred sharehold...,"2,066","2,174","2,122","4,240","3,843"


### Table 3

In [108]:
# Extract tables from the PDF (specify the page number)
tables = tabula.read_pdf(pdf_path, pages=8, multiple_tables=False)

# Convert the first table in the list to a DataFrame
df3 = pd.DataFrame(tables[0])
df3

Aug 03, 2024 6:41:29 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 03, 2024 6:41:33 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Aug 03, 2024 6:41:33 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,International,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),Banking(2),Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,008",$ 695,$ 382 $,428,$,(421),$,"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,24,2,–,NaN,–,NaN,26
5,Reported net income attributable to equity hol...,NaN,"1,008",671,380,428,NaN,(421),NaN,"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,–,–,–,–,NaN,123,NaN,123
8,Reported net income attributable to common sha...,$,"1,008",$ 671,$ 380 $,428,$,(544),$,"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df3.drop(columns=['Unnamed: 1','Unnamed: 6','Unnamed: 8'], inplace=True)
df3

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 7,Unnamed: 9
0,NaN,Canadian,International,Wealth,Banking and,NaN,NaN
1,($ millions),Banking(2),Banking(2),Management,Markets,Other,Total(2)
2,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),–,24,2,–,–,26
5,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,–,–,–,–,123,123
8,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df3.drop(index=[0, 1], inplace=True)
df3

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 7,Unnamed: 9
2,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),–,24,2,–,–,26
5,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,–,–,–,–,123,123
8,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
10,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
11,Amortization of acquisition-related intangible...,1,8,9,–,–,18


In [111]:
df3.rename(columns={
    'Unnamed: 0': '$millions',
    'Unnamed: 2': 'Canadian banking',
    'Unnamed: 3': 'International Banking',
    'Global': 'Global Wealth Management',
    'Global.1': 'Global Banking and Markets',
    'Unnamed: 5': 'Other',
    'Unnamed: 7': 'Total'
}, inplace=True)
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
2,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),–,24,2,–,–,26
5,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,–,–,–,–,123,123
8,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
10,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
11,Amortization of acquisition-related intangible...,1,8,9,–,–,18


In [112]:
df3 = df3.iloc[:19].reset_index(drop=True)
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
0,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,24,2,–,–,26
3,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,–,–,–,–,123,123
6,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,1,8,9,–,–,18


In [113]:
df3.drop(index=1, inplace=True)
df3.drop(index=4, inplace=True)
df3.drop(index=8, inplace=True)
df3.drop(index=14, inplace=True)
df3.reset_index(drop=True, inplace=True)
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
0,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
1,subsidiaries (NCI),–,24,2,–,–,26
2,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
3,shareholders and other equity instrument holders,–,–,–,–,123,123
4,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
5,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
6,Amortization of acquisition-related intangible...,1,8,9,–,–,18
7,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
8,Total impact of adjusting items on net income ...,1,8,9,–,–,18
9,Impact of adjusting items on income tax expense,(1),(2),(2),–,–,(5)


In [114]:
# Rename row 11 (index 10) value

df3.iloc[1, df3.columns.get_loc('$millions')] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df3.iloc[3, df3.columns.get_loc('$millions')] = 'Reported net income attributable to preferred shareholders and other equity instrument holders '
df3.iloc[6, df3.columns.get_loc('$millions')] = 'Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets '
df3.iloc[11, df3.columns.get_loc('$millions')] = 'Total impact of adjusting items on net income attributable to equity holders and common shareholders'
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
0,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
1,Net income attributable to non-controlling int...,–,24,2,–,–,26
2,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
3,Reported net income attributable to preferred ...,–,–,–,–,123,123
4,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
5,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
6,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
7,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
8,Total impact of adjusting items on net income ...,1,8,9,–,–,18
9,Impact of adjusting items on income tax expense,(1),(2),(2),–,–,(5)


In [124]:
df3.drop(index=5, inplace=True)
df3.reset_index(drop=True, inplace=True)
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
0,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
1,Net income attributable to non-controlling int...,–,24,2,–,–,26
2,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
3,Reported net income attributable to preferred ...,–,–,–,–,123,123
4,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
5,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
6,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
7,Total impact of adjusting items on net income ...,1,8,9,–,–,18
8,Impact of adjusting items on income tax expense,(1),(2),(2),–,–,(5)
9,Total impact of adjusting items on net income,–,6,7,–,–,13


#### Cleaned Table 3

In [125]:
df3

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Total,Unnamed: 9
0,Reported net income (loss),"1,008",$ 695,$ 382 $,428,(421),"2,092"
1,Net income attributable to non-controlling int...,–,24,2,–,–,26
2,Reported net income attributable to equity hol...,"1,008",671,380,428,(421),"2,066"
3,Reported net income attributable to preferred ...,–,–,–,–,123,123
4,Reported net income attributable to common sha...,"1,008",$ 671,$ 380 $,428,(544),"1,943"
5,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
6,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
7,Total impact of adjusting items on net income ...,1,8,9,–,–,18
8,Impact of adjusting items on income tax expense,(1),(2),(2),–,–,(5)
9,Total impact of adjusting items on net income,–,6,7,–,–,13


### Table 4

In [115]:
# Extract tables from the PDF (specify the page number)
tables = tabula.read_pdf(pdf_path, pages=8, multiple_tables=False)

# Convert the first table in the list to a DataFrame
df4 = pd.DataFrame(tables[0])
df4

Aug 03, 2024 6:45:13 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 03, 2024 6:45:15 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Aug 03, 2024 6:45:15 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,International,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),Banking(2),Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,008",$ 695,$ 382 $,428,$,(421),$,"2,092"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,24,2,–,NaN,–,NaN,26
5,Reported net income attributable to equity hol...,NaN,"1,008",671,380,428,NaN,(421),NaN,"2,066"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,–,–,–,–,NaN,123,NaN,123
8,Reported net income attributable to common sha...,$,"1,008",$ 671,$ 380 $,428,$,(544),$,"1,943"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df4 = df4.iloc[28:].reset_index(drop=True)
df4

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,International,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),Banking(2),Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,095",$ 768,$ 371 $,439,$,(474),$,"2,199"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,22,3,–,NaN,–,NaN,25
5,Reported net income attributable to equity hol...,NaN,"1,095",746,368,439,NaN,(474),NaN,"2,174"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,1,1,–,1,NaN,105,NaN,108
8,Reported net income attributable to common sha...,$,"1,094",$ 745,$ 368 $,438,$,(579),$,"2,066"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
df4.drop(index=[0, 1], inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Reported net income (loss),$,"1,095",$ 768,$ 371 $,439,$,(474),$,"2,199"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,22,3,–,NaN,–,NaN,25
3,Reported net income attributable to equity hol...,NaN,"1,095",746,368,439,NaN,(474),NaN,"2,174"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,1,1,–,1,NaN,105,NaN,108
6,Reported net income attributable to common sha...,$,"1,094",$ 745,$ 368 $,438,$,(579),$,"2,066"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,1,8,9,–,NaN,–,NaN,18


In [118]:
df4.drop(columns=['Unnamed: 1','Unnamed: 6','Unnamed: 8'], inplace=True)
df4

,Unnamed: 0,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 7,Unnamed: 9
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,22,3,–,–,25
3,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,1,1,–,1,105,108
6,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,1,8,9,–,–,18


In [119]:
df4.rename(columns={
    'Unnamed: 0': '$millions',
    'Unnamed: 2': 'Canadian banking',
    'Unnamed: 3': 'International Banking',
    'Global': 'Global Wealth Management',
    'Global.1': 'Global Banking and Markets',
    'Unnamed: 7': 'Other',
    'Unnamed: 9': 'Total'
}, inplace=True)
df4


,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,22,3,–,–,25
3,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,1,1,–,1,105,108
6,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,1,8,9,–,–,18


In [120]:
df4.drop(index=1, inplace=True)
df4.drop(index=4, inplace=True)
df4.drop(index=8, inplace=True)
df4.drop(index=14, inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,subsidiaries (NCI),–,22,3,–,–,25
2,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
3,shareholders and other equity instrument holders,1,1,–,1,105,108
4,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
5,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
6,Amortization of acquisition-related intangible...,1,8,9,–,–,18
7,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
8,Total impact of adjusting items on net income ...,1,8,9,–,–,18
9,Impact of adjusting items on income tax expense,–,(2),(3),–,–,(5)


In [121]:
# Rename row 11 (index 10) value

df4.iloc[1, df4.columns.get_loc('$millions')] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df4.iloc[3, df4.columns.get_loc('$millions')] = 'Reported net income attributable to preferred shareholders and other equity instrument holders '
df4.iloc[6, df4.columns.get_loc('$millions')] = 'Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets '
df4.iloc[11, df4.columns.get_loc('$millions')] = 'Total impact of adjusting items on net income attributable to equity holders and common shareholders'
df4

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,Net income attributable to non-controlling int...,–,22,3,–,–,25
2,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
3,Reported net income attributable to preferred ...,1,1,–,1,105,108
4,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
5,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
6,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
7,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
8,Total impact of adjusting items on net income ...,1,8,9,–,–,18
9,Impact of adjusting items on income tax expense,–,(2),(3),–,–,(5)


In [122]:
df4.drop(index=5, inplace=True)
df4.reset_index(drop=True, inplace=True)
df4

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,Net income attributable to non-controlling int...,–,22,3,–,–,25
2,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
3,Reported net income attributable to preferred ...,1,1,–,1,105,108
4,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
5,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
6,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
7,Total impact of adjusting items on net income ...,1,8,9,–,–,18
8,Impact of adjusting items on income tax expense,–,(2),(3),–,–,(5)
9,Total impact of adjusting items on net income,1,6,6,–,–,13


#### Cleaned Table 4

In [123]:
df4

,$millions,Canadian banking,International Banking,Global Wealth Management,Global Banking and Markets,Other,Total
0,Reported net income (loss),"1,095",$ 768,$ 371 $,439,(474),"2,199"
1,Net income attributable to non-controlling int...,–,22,3,–,–,25
2,Reported net income attributable to equity hol...,"1,095",746,368,439,(474),"2,174"
3,Reported net income attributable to preferred ...,1,1,–,1,105,108
4,Reported net income attributable to common sha...,"1,094",$ 745,$ 368 $,438,(579),"2,066"
5,Adjusting items impacting non-interest expense...,1,8,9,–,–,18
6,Total non-interest expenses adjustments (Pre-tax),1,8,9,–,–,18
7,Total impact of adjusting items on net income ...,1,8,9,–,–,18
8,Impact of adjusting items on income tax expense,–,(2),(3),–,–,(5)
9,Total impact of adjusting items on net income,1,6,6,–,–,13


### Table5

In [2]:
!pip install tabula-py

import tabula
import pandas as pd

# Path to the PDF file
pdf_path = "/content/Data.pdf"

# Extract tables from the PDF on page 9
tables = tabula.read_pdf(pdf_path, pages=9, multiple_tables=True)

# Convert the first table in the list to a DataFrame and limit to the first 27 columns
df5 = pd.DataFrame(tables[0])
df5 = df5.iloc[:21]

# Display the DataFrame
df5

Aug 03, 2024 5:34:17 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Aug 03, 2024 5:34:17 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Aug 03, 2024 5:34:18 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Aug 03, 2024 5:34:18 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Aug 03, 2024 5:34:23 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Aug 03, 2024 5:34:23 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,NaN,International,NaN,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),NaN,Banking(2),NaN,Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"1,055",$,657,$,356 $,401,$,(323),$,"2,146"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,NaN,21,NaN,3,–,NaN,–,NaN,24
5,Reported net income attributable to equity hol...,NaN,"1,055",NaN,636,NaN,353,401,NaN,(323),NaN,"2,122"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,1,1,NaN,100,NaN,104
8,Reported net income attributable to common sha...,$,"1,054",$,635,$,352 $,400,$,(423),$,"2,018"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df5.drop(index=[0, 1], inplace=True)
df5.reset_index(drop=True, inplace=True)
df5

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Reported net income (loss),$,"1,055",$,657,$,356 $,401,$,(323),$,"2,146"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,21,NaN,3,–,NaN,–,NaN,24
3,Reported net income attributable to equity hol...,NaN,"1,055",NaN,636,NaN,353,401,NaN,(323),NaN,"2,122"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,1,1,NaN,100,NaN,104
6,Reported net income attributable to common sha...,$,"1,054",$,635,$,352 $,400,$,(423),$,"2,018"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,1,NaN,11,NaN,9,–,NaN,–,NaN,21


In [4]:
df5.rename(columns={
    'Unnamed: 0': '$millions',
    'Unnamed: 2': 'Canadian banking(For the three months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 4': 'International Banking(For the three months ended April 30, 2023⁽¹⁾)',
    'Global': 'Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)',
    'Global.1': 'Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 7': 'Other(For the three months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 9': 'Total(For the three months ended April 30, 2023⁽¹⁾)'
}, inplace=True)
df5

,$millions,Unnamed: 1,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)",Unnamed: 3,"International Banking(For the three months ended April 30, 2023⁽¹⁾)",Unnamed: 5,"Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)",Unnamed: 6,"Other(For the three months ended April 30, 2023⁽¹⁾)",Unnamed: 8,"Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),$,"1,055",$,657,$,356 $,401,$,(323),$,"2,146"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,21,NaN,3,–,NaN,–,NaN,24
3,Reported net income attributable to equity hol...,NaN,"1,055",NaN,636,NaN,353,401,NaN,(323),NaN,"2,122"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,1,1,NaN,100,NaN,104
6,Reported net income attributable to common sha...,$,"1,054",$,635,$,352 $,400,$,(423),$,"2,018"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,1,NaN,11,NaN,9,–,NaN,–,NaN,21


In [5]:
df5.drop(columns=['Unnamed: 1','Unnamed: 3','Unnamed: 5','Unnamed: 6', 'Unnamed: 8'], inplace=True)
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356 $,401,(323),"2,146"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,21,3,–,–,24
3,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,1,1,1,1,100,104
6,Reported net income attributable to common sha...,"1,054",635,352 $,400,(423),"2,018"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,1,11,9,–,–,21


In [6]:
df5.drop(index=1, inplace=True)
df5.drop(index=4, inplace=True)
df5.drop(index=7, inplace=True)
df5.drop(index=8, inplace=True)
df5.drop(index=14, inplace=True)
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356 $,401,(323),"2,146"
2,subsidiaries (NCI),–,21,3,–,–,24
3,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
5,shareholders and other equity instrument holders,1,1,1,1,100,104
6,Reported net income attributable to common sha...,"1,054",635,352 $,400,(423),"2,018"
9,Amortization of acquisition-related intangible...,1,11,9,–,–,21
10,Total non-interest expenses adjustments (Pre-tax),1,11,9,–,–,21
11,Total impact of adjusting items on net income ...,1,11,9,–,–,21
12,Impact of adjusting items on income tax expense,–,(3),(3),–,–,(6)
13,Total impact of adjusting items on net income,1,8,6,–,–,15


In [7]:
df5.reset_index(drop=True, inplace=True)
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356 $,401,(323),"2,146"
1,subsidiaries (NCI),–,21,3,–,–,24
2,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
3,shareholders and other equity instrument holders,1,1,1,1,100,104
4,Reported net income attributable to common sha...,"1,054",635,352 $,400,(423),"2,018"
5,Amortization of acquisition-related intangible...,1,11,9,–,–,21
6,Total non-interest expenses adjustments (Pre-tax),1,11,9,–,–,21
7,Total impact of adjusting items on net income ...,1,11,9,–,–,21
8,Impact of adjusting items on income tax expense,–,(3),(3),–,–,(6)
9,Total impact of adjusting items on net income,1,8,6,–,–,15


In [8]:
# Renaming the rows name

df5.iloc[1, df5.columns.get_loc('$millions')] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df5.iloc[3, df5.columns.get_loc('$millions')] = 'Reported net income attributable to preferred shareholders and other equity instrument holders'
df5.iloc[5, df5.columns.get_loc('$millions')] = 'Adjustments: Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets '
df5.iloc[10, df5.columns.get_loc('$millions')] = 'Total impact of adjusting items on net income attributable to equity holders and common shareholders'
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356 $,401,(323),"2,146"
1,Net income attributable to non-controlling int...,–,21,3,–,–,24
2,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
3,Reported net income attributable to preferred ...,1,1,1,1,100,104
4,Reported net income attributable to common sha...,"1,054",635,352 $,400,(423),"2,018"
5,Adjustments: Adjusting items impacting non-int...,1,11,9,–,–,21
6,Total non-interest expenses adjustments (Pre-tax),1,11,9,–,–,21
7,Total impact of adjusting items on net income ...,1,11,9,–,–,21
8,Impact of adjusting items on income tax expense,–,(3),(3),–,–,(6)
9,Total impact of adjusting items on net income,1,8,6,–,–,15


In [9]:
# Remove the $ sign from the (Taxable equivalent basis)
df5['Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)'] = df5['Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)'].str.replace('$', '', regex=False)
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356,401,(323),"2,146"
1,Net income attributable to non-controlling int...,–,21,3,–,–,24
2,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
3,Reported net income attributable to preferred ...,1,1,1,1,100,104
4,Reported net income attributable to common sha...,"1,054",635,352,400,(423),"2,018"
5,Adjustments: Adjusting items impacting non-int...,1,11,9,–,–,21
6,Total non-interest expenses adjustments (Pre-tax),1,11,9,–,–,21
7,Total impact of adjusting items on net income ...,1,11,9,–,–,21
8,Impact of adjusting items on income tax expense,–,(3),(3),–,–,(6)
9,Total impact of adjusting items on net income,1,8,6,–,–,15


<h1 id = "table5"></h1?
#### Cleaned Table 5

#### Cleaned table 5

In [10]:
df5

,$millions,"Canadian banking(For the three months ended April 30, 2023⁽¹⁾)","International Banking(For the three months ended April 30, 2023⁽¹⁾)","Global Wealth Management(For the three months ended April 30, 2023⁽¹⁾)","Global Banking and Markets(For the three months ended April 30, 2023⁽¹⁾)","Other(For the three months ended April 30, 2023⁽¹⁾)","Total(For the three months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"1,055",657,356,401,(323),"2,146"
1,Net income attributable to non-controlling int...,–,21,3,–,–,24
2,Reported net income attributable to equity hol...,"1,055",636,353,401,(323),"2,122"
3,Reported net income attributable to preferred ...,1,1,1,1,100,104
4,Reported net income attributable to common sha...,"1,054",635,352,400,(423),"2,018"
5,Adjustments: Adjusting items impacting non-int...,1,11,9,–,–,21
6,Total non-interest expenses adjustments (Pre-tax),1,11,9,–,–,21
7,Total impact of adjusting items on net income ...,1,11,9,–,–,21
8,Impact of adjusting items on income tax expense,–,(3),(3),–,–,(6)
9,Total impact of adjusting items on net income,1,8,6,–,–,15


### Table 6

In [11]:
# Convert the first table in the list to a DataFrame
df = pd.DataFrame(tables[0])

# Select all rows starting from the 28th row and reset the index
df6 = df.iloc[29:].reset_index(drop=True)

# Set the 28th row (now the first row in df6) as the header
df6.columns = df6.iloc[0]
df6 = df6[1:]
df6

,($ millions),Banking(2),NaN,NaN,Banking(2),Management,NaN,Markets,NaN,Other,NaN,Total(2)
1,Reported net income (loss),$,"2,103",$,"1,463 $",NaN,753 $,867,$,(895),$,"4,291"
2,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subsidiaries (NCI),NaN,–,NaN,46,NaN,5,–,NaN,–,NaN,51
4,Reported net income attributable to equity hol...,NaN,"2,103",NaN,"1,417",NaN,748,867,NaN,(895),NaN,"4,240"
5,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,–,1,NaN,228,NaN,231
7,Reported net income attributable to common sha...,$,"2,102",$,"1,416 $",NaN,748 $,866,$,"(1,123)",$,"4,009"
8,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Amortization of acquisition-related intangible...,NaN,2,NaN,16,NaN,18,–,NaN,–,NaN,36


In [12]:
df6 = df.iloc[29:].reset_index(drop=True)
df6.reset_index(drop=True, inplace=True)
df6

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,($ millions),Banking(2),NaN,NaN,Banking(2),Management,NaN,Markets,NaN,Other,NaN,Total(2)
1,Reported net income (loss),$,"2,103",$,"1,463 $",NaN,753 $,867,$,(895),$,"4,291"
2,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,subsidiaries (NCI),NaN,–,NaN,46,NaN,5,–,NaN,–,NaN,51
4,Reported net income attributable to equity hol...,NaN,"2,103",NaN,"1,417",NaN,748,867,NaN,(895),NaN,"4,240"
5,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,–,1,NaN,228,NaN,231
7,Reported net income attributable to common sha...,$,"2,102",$,"1,416 $",NaN,748 $,866,$,"(1,123)",$,"4,009"
8,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df6.drop(index=[0], inplace=True)
df6.reset_index(drop=True, inplace=True)


In [14]:
df6

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Reported net income (loss),$,"2,103",$,"1,463 $",NaN,753 $,867,$,(895),$,"4,291"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,46,NaN,5,–,NaN,–,NaN,51
3,Reported net income attributable to equity hol...,NaN,"2,103",NaN,"1,417",NaN,748,867,NaN,(895),NaN,"4,240"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,–,1,NaN,228,NaN,231
6,Reported net income attributable to common sha...,$,"2,102",$,"1,416 $",NaN,748 $,866,$,"(1,123)",$,"4,009"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,2,NaN,16,NaN,18,–,NaN,–,NaN,36


In [15]:
df6.rename(columns={
    'Unnamed: 0': '$millions',
    'Unnamed: 2': 'Canadian banking (For the six months ended April 30, 2024⁽¹⁾)',
    'Unnamed: 4': 'International Banking (For the six months ended April 30, 2024⁽¹⁾)',
    'Global': 'Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)',
    'Global.1': 'Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)',
    'Unnamed: 7': 'Other (For the six months ended April 30, 2024⁽¹⁾)',
    'Unnamed: 9': 'Total (For the six months ended April 30, 2024⁽¹⁾)'
}, inplace=True)
df6

,$millions,Unnamed: 1,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)",Unnamed: 3,"International Banking (For the six months ended April 30, 2024⁽¹⁾)",Unnamed: 5,"Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)",Unnamed: 6,"Other (For the six months ended April 30, 2024⁽¹⁾)",Unnamed: 8,"Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),$,"2,103",$,"1,463 $",NaN,753 $,867,$,(895),$,"4,291"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,46,NaN,5,–,NaN,–,NaN,51
3,Reported net income attributable to equity hol...,NaN,"2,103",NaN,"1,417",NaN,748,867,NaN,(895),NaN,"4,240"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,1,NaN,1,NaN,–,1,NaN,228,NaN,231
6,Reported net income attributable to common sha...,$,"2,102",$,"1,416 $",NaN,748 $,866,$,"(1,123)",$,"4,009"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,2,NaN,16,NaN,18,–,NaN,–,NaN,36


In [16]:
df6.drop(columns=['Unnamed: 1','Unnamed: 3','Unnamed: 5','Unnamed: 6','Unnamed: 8'], inplace=True)
df6

,$millions,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)","International Banking (For the six months ended April 30, 2024⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)","Other (For the six months ended April 30, 2024⁽¹⁾)","Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),"2,103","1,463 $",753 $,867,(895),"4,291"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,46,5,–,–,51
3,Reported net income attributable to equity hol...,"2,103","1,417",748,867,(895),"4,240"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,1,1,–,1,228,231
6,Reported net income attributable to common sha...,"2,102","1,416 $",748 $,866,"(1,123)","4,009"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,2,16,18,–,–,36


In [17]:
df6.drop(index=1, inplace=True)
df6.drop(index=4, inplace=True)
df6.drop(index=7, inplace=True)
df6.drop(index=8, inplace=True)
df6.drop(index=14, inplace=True)
df6.reset_index(drop=True, inplace=True)
df6

,$millions,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)","International Banking (For the six months ended April 30, 2024⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)","Other (For the six months ended April 30, 2024⁽¹⁾)","Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),"2,103","1,463 $",753 $,867,(895),"4,291"
1,subsidiaries (NCI),–,46,5,–,–,51
2,Reported net income attributable to equity hol...,"2,103","1,417",748,867,(895),"4,240"
3,shareholders and other equity instrument holders,1,1,–,1,228,231
4,Reported net income attributable to common sha...,"2,102","1,416 $",748 $,866,"(1,123)","4,009"
5,Amortization of acquisition-related intangible...,2,16,18,–,–,36
6,Total non-interest expenses adjustments (Pre-tax),2,16,18,–,–,36
7,Total impact of adjusting items on net income ...,2,16,18,–,–,36
8,Impact of adjusting items on income tax expense,(1),(4),(5),–,–,(10)
9,Total impact of adjusting items on net income,1,12,13,–,–,26


In [18]:
df6.iloc[1, df6.columns.get_loc('$millions')] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df6.iloc[3, df6.columns.get_loc('$millions')] = 'Reported net income attributable to preferred shareholders and other equity instrument holders'
df6.iloc[5, df6.columns.get_loc('$millions')] = 'Adjustments: Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets '
df6.iloc[10, df6.columns.get_loc('$millions')] = 'Total impact of adjusting items on net income attributable to equity holders and common shareholders'
df6

,$millions,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)","International Banking (For the six months ended April 30, 2024⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)","Other (For the six months ended April 30, 2024⁽¹⁾)","Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),"2,103","1,463 $",753 $,867,(895),"4,291"
1,Net income attributable to non-controlling int...,–,46,5,–,–,51
2,Reported net income attributable to equity hol...,"2,103","1,417",748,867,(895),"4,240"
3,Reported net income attributable to preferred ...,1,1,–,1,228,231
4,Reported net income attributable to common sha...,"2,102","1,416 $",748 $,866,"(1,123)","4,009"
5,Adjustments: Adjusting items impacting non-int...,2,16,18,–,–,36
6,Total non-interest expenses adjustments (Pre-tax),2,16,18,–,–,36
7,Total impact of adjusting items on net income ...,2,16,18,–,–,36
8,Impact of adjusting items on income tax expense,(1),(4),(5),–,–,(10)
9,Total impact of adjusting items on net income,1,12,13,–,–,26


In [19]:
# Remove the $ sign from the (The 2 columns)
df6['International Banking (For the six months ended April 30, 2024⁽¹⁾)'] = df6['International Banking (For the six months ended April 30, 2024⁽¹⁾)'].str.replace('$', '', regex=False)
df6['Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)'] = df6['Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)'].str.replace('$', '', regex=False)
df6

,$millions,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)","International Banking (For the six months ended April 30, 2024⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)","Other (For the six months ended April 30, 2024⁽¹⁾)","Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),"2,103","1,463",753,867,(895),"4,291"
1,Net income attributable to non-controlling int...,–,46,5,–,–,51
2,Reported net income attributable to equity hol...,"2,103","1,417",748,867,(895),"4,240"
3,Reported net income attributable to preferred ...,1,1,–,1,228,231
4,Reported net income attributable to common sha...,"2,102","1,416",748,866,"(1,123)","4,009"
5,Adjustments: Adjusting items impacting non-int...,2,16,18,–,–,36
6,Total non-interest expenses adjustments (Pre-tax),2,16,18,–,–,36
7,Total impact of adjusting items on net income ...,2,16,18,–,–,36
8,Impact of adjusting items on income tax expense,(1),(4),(5),–,–,(10)
9,Total impact of adjusting items on net income,1,12,13,–,–,26


#### Cleaned Table 6

In [20]:
df6

,$millions,"Canadian banking (For the six months ended April 30, 2024⁽¹⁾)","International Banking (For the six months ended April 30, 2024⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2024⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2024⁽¹⁾)","Other (For the six months ended April 30, 2024⁽¹⁾)","Total (For the six months ended April 30, 2024⁽¹⁾)"
0,Reported net income (loss),"2,103","1,463",753,867,(895),"4,291"
1,Net income attributable to non-controlling int...,–,46,5,–,–,51
2,Reported net income attributable to equity hol...,"2,103","1,417",748,867,(895),"4,240"
3,Reported net income attributable to preferred ...,1,1,–,1,228,231
4,Reported net income attributable to common sha...,"2,102","1,416",748,866,"(1,123)","4,009"
5,Adjustments: Adjusting items impacting non-int...,2,16,18,–,–,36
6,Total non-interest expenses adjustments (Pre-tax),2,16,18,–,–,36
7,Total impact of adjusting items on net income ...,2,16,18,–,–,36
8,Impact of adjusting items on income tax expense,(1),(4),(5),–,–,(10)
9,Total impact of adjusting items on net income,1,12,13,–,–,26


### Table 7

In [21]:
# Path to the PDF file
pdf_path = "/content/Data.pdf"

# Extract tables from the PDF on page 10
tables = tabula.read_pdf(pdf_path, pages=10, multiple_tables=True)

# Convert the first table in the list to a DataFrame and limit to the first 27 columns
df7 = pd.DataFrame(tables[0])
df7 = df7.iloc[:24]
df7



Aug 03, 2024 5:34:37 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Canadian,NaN,International,NaN,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),NaN,Banking(2),NaN,Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"2,141",$,"1,336",$,743 $,920,$,"(1,236)",$,"3,904"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,NaN,56,NaN,5,–,NaN,–,NaN,61
5,Reported net income attributable to equity hol...,NaN,"2,141",NaN,"1,280",NaN,738,920,NaN,"(1,236)",NaN,"3,843"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,2,NaN,2,NaN,1,2,NaN,198,NaN,205
8,Reported net income attributable to common sha...,$,"2,139",$,"1,278",$,737 $,918,$,"(1,434)",$,"3,638"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df7.drop(index=[0, 1], inplace=True)
df7.reset_index(drop=True, inplace=True)
df7

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Reported net income (loss),$,"2,141",$,"1,336",$,743 $,920,$,"(1,236)",$,"3,904"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,56,NaN,5,–,NaN,–,NaN,61
3,Reported net income attributable to equity hol...,NaN,"2,141",NaN,"1,280",NaN,738,920,NaN,"(1,236)",NaN,"3,843"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,2,NaN,2,NaN,1,2,NaN,198,NaN,205
6,Reported net income attributable to common sha...,$,"2,139",$,"1,278",$,737 $,918,$,"(1,434)",$,"3,638"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,3,NaN,21,NaN,18,–,NaN,–,NaN,42


In [23]:
df7.rename(columns={
    'Unnamed: 0': '$millions',
    'Unnamed: 2': 'Canadian banking (For the six months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 4': 'International Banking (For the six months ended April 30, 2023⁽¹⁾)',
    'Global': 'Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)',
    'Global.1': 'Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 7': 'Other (For the six months ended April 30, 2023⁽¹⁾)',
    'Unnamed: 9': 'Total (For the six months ended April 30, 2023⁽¹⁾)'
}, inplace=True)
df7

,$millions,Unnamed: 1,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)",Unnamed: 3,"International Banking (For the six months ended April 30, 2023⁽¹⁾)",Unnamed: 5,"Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)",Unnamed: 6,"Other (For the six months ended April 30, 2023⁽¹⁾)",Unnamed: 8,"Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),$,"2,141",$,"1,336",$,743 $,920,$,"(1,236)",$,"3,904"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),NaN,–,NaN,56,NaN,5,–,NaN,–,NaN,61
3,Reported net income attributable to equity hol...,NaN,"2,141",NaN,"1,280",NaN,738,920,NaN,"(1,236)",NaN,"3,843"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,NaN,2,NaN,2,NaN,1,2,NaN,198,NaN,205
6,Reported net income attributable to common sha...,$,"2,139",$,"1,278",$,737 $,918,$,"(1,434)",$,"3,638"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,NaN,3,NaN,21,NaN,18,–,NaN,–,NaN,42


In [24]:
df7.drop(columns=['Unnamed: 1','Unnamed: 3','Unnamed: 5','Unnamed: 6', 'Unnamed: 8'], inplace=True)
df7

,$millions,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)","International Banking (For the six months ended April 30, 2023⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)","Other (For the six months ended April 30, 2023⁽¹⁾)","Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"2,141","1,336",743 $,920,"(1,236)","3,904"
1,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN
2,subsidiaries (NCI),–,56,5,–,–,61
3,Reported net income attributable to equity hol...,"2,141","1,280",738,920,"(1,236)","3,843"
4,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN
5,shareholders and other equity instrument holders,2,2,1,2,198,205
6,Reported net income attributable to common sha...,"2,139","1,278",737 $,918,"(1,434)","3,638"
7,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN
8,Adjusting items impacting non-interest expense...,NaN,NaN,NaN,NaN,NaN,NaN
9,Amortization of acquisition-related intangible...,3,21,18,–,–,42


In [25]:
df7.drop(index=1, inplace=True)
df7.drop(index=4, inplace=True)
df7.drop(index=7, inplace=True)
df7.drop(index=8, inplace=True)
df7.drop(index=12, inplace=True)
df7.drop(index=17, inplace=True)
df7.reset_index(drop=True, inplace=True)
df7

,$millions,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)","International Banking (For the six months ended April 30, 2023⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)","Other (For the six months ended April 30, 2023⁽¹⁾)","Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"2,141","1,336",743 $,920,"(1,236)","3,904"
1,subsidiaries (NCI),–,56,5,–,–,61
2,Reported net income attributable to equity hol...,"2,141","1,280",738,920,"(1,236)","3,843"
3,shareholders and other equity instrument holders,2,2,1,2,198,205
4,Reported net income attributable to common sha...,"2,139","1,278",737 $,918,"(1,434)","3,638"
5,Amortization of acquisition-related intangible...,3,21,18,–,–,42
6,Total non-interest expenses adjustments (Pre-tax),3,21,18,–,–,42
7,Total impact of adjusting items on net income ...,3,21,18,–,–,42
8,Canada recovery dividend,–,–,–,–,579,579
9,Impact of other adjusting items on income tax ...,(1),(6),(5),–,–,(12)


In [26]:
df7.iloc[1, df7.columns.get_loc('$millions')] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df7.iloc[3, df7.columns.get_loc('$millions')] = 'Reported net income attributable to preferred shareholders and other equity instrument holders'
df7.iloc[5, df7.columns.get_loc('$millions')] = 'Adjustments: Adjusting items impacting non-interest expenses (Pre-tax) Amortization of acquisition-related intangible assets '
df7.iloc[8, df7.columns.get_loc('$millions')] = 'Impact of adjusting items on income tax expense Canada recovery dividend'
df7.iloc[12, df7.columns.get_loc('$millions')] = 'Total impact of adjusting items on net income attributableto equity holders and common shareholders'
df7

,$millions,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)","International Banking (For the six months ended April 30, 2023⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)","Other (For the six months ended April 30, 2023⁽¹⁾)","Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"2,141","1,336",743 $,920,"(1,236)","3,904"
1,Net income attributable to non-controlling int...,–,56,5,–,–,61
2,Reported net income attributable to equity hol...,"2,141","1,280",738,920,"(1,236)","3,843"
3,Reported net income attributable to preferred ...,2,2,1,2,198,205
4,Reported net income attributable to common sha...,"2,139","1,278",737 $,918,"(1,434)","3,638"
5,Adjustments: Adjusting items impacting non-int...,3,21,18,–,–,42
6,Total non-interest expenses adjustments (Pre-tax),3,21,18,–,–,42
7,Total impact of adjusting items on net income ...,3,21,18,–,–,42
8,Impact of adjusting items on income tax expens...,–,–,–,–,579,579
9,Impact of other adjusting items on income tax ...,(1),(6),(5),–,–,(12)


In [27]:
df7['Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)'] = df7['Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)'].str.replace('$', '', regex=False)
df7

,$millions,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)","International Banking (For the six months ended April 30, 2023⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)","Other (For the six months ended April 30, 2023⁽¹⁾)","Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"2,141","1,336",743,920,"(1,236)","3,904"
1,Net income attributable to non-controlling int...,–,56,5,–,–,61
2,Reported net income attributable to equity hol...,"2,141","1,280",738,920,"(1,236)","3,843"
3,Reported net income attributable to preferred ...,2,2,1,2,198,205
4,Reported net income attributable to common sha...,"2,139","1,278",737,918,"(1,434)","3,638"
5,Adjustments: Adjusting items impacting non-int...,3,21,18,–,–,42
6,Total non-interest expenses adjustments (Pre-tax),3,21,18,–,–,42
7,Total impact of adjusting items on net income ...,3,21,18,–,–,42
8,Impact of adjusting items on income tax expens...,–,–,–,–,579,579
9,Impact of other adjusting items on income tax ...,(1),(6),(5),–,–,(12)


#### Cleaned Table 7

In [28]:
df7

,$millions,"Canadian banking (For the six months ended April 30, 2023⁽¹⁾)","International Banking (For the six months ended April 30, 2023⁽¹⁾)","Global Wealth Management (For the six months ended April 30, 2023⁽¹⁾)","Global Banking and Markets (For the six months ended April 30, 2023⁽¹⁾)","Other (For the six months ended April 30, 2023⁽¹⁾)","Total (For the six months ended April 30, 2023⁽¹⁾)"
0,Reported net income (loss),"2,141","1,336",743,920,"(1,236)","3,904"
1,Net income attributable to non-controlling int...,–,56,5,–,–,61
2,Reported net income attributable to equity hol...,"2,141","1,280",738,920,"(1,236)","3,843"
3,Reported net income attributable to preferred ...,2,2,1,2,198,205
4,Reported net income attributable to common sha...,"2,139","1,278",737,918,"(1,434)","3,638"
5,Adjustments: Adjusting items impacting non-int...,3,21,18,–,–,42
6,Total non-interest expenses adjustments (Pre-tax),3,21,18,–,–,42
7,Total impact of adjusting items on net income ...,3,21,18,–,–,42
8,Impact of adjusting items on income tax expens...,–,–,–,–,579,579
9,Impact of other adjusting items on income tax ...,(1),(6),(5),–,–,(12)


### Table 8

In [29]:
# Extract tables from the PDF (specify the page number)
tables = tabula.read_pdf(pdf_path, pages=10, multiple_tables=False)

# Convert the first table in the list to a DataFrame
df8 = pd.DataFrame(tables[0])
df8

Aug 03, 2024 5:34:49 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Global,Global.1,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,NaN,Canadian,NaN,International,NaN,Wealth,Banking and,NaN,NaN,NaN,NaN
1,($ millions),NaN,Banking(2),NaN,Banking(2),NaN,Management,Markets,NaN,Other,NaN,Total(2)
2,Reported net income (loss),$,"2,141",$,"1,336",$,743 $,920,$,"(1,236)",$,"3,904"
3,Net income attributable to non-controlling int...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,subsidiaries (NCI),NaN,–,NaN,56,NaN,5,–,NaN,–,NaN,61
5,Reported net income attributable to equity hol...,NaN,"2,141",NaN,"1,280",NaN,738,920,NaN,"(1,236)",NaN,"3,843"
6,Reported net income attributable to preferred,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,shareholders and other equity instrument holders,NaN,2,NaN,2,NaN,1,2,NaN,198,NaN,205
8,Reported net income attributable to common sha...,$,"2,139",$,"1,278",$,737 $,918,$,"(1,434)",$,"3,638"
9,Adjustments:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df8 = df8.iloc[34:]

In [31]:
df8 = df8.drop(columns=['Unnamed: 1','Unnamed: 5','Unnamed: 10','Unnamed: 8'])
df8 = df8.iloc[5:].reset_index(drop=True)
df8


,Unnamed: 0,Unnamed: 2,Unnamed: 3,Unnamed: 4,Global,Global.1,Unnamed: 9,Unnamed: 11
0,"Net interest income $ 2,246",19 $,NaN,"2,227 $ 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income 857,6,NaN,851 743,(88),"831 1,535",(163),"1,698"
2,"Total revenue 3,103",25,"3,078","2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses 574,6,NaN,568 436,(3),439 840,(27),867
4,"Non-interest expenses 1,571",2,NaN,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense 190,4,NaN,186 171,(10),181 339,(20),359
6,Net income $ 768,13 $,NaN,755 $ 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,Net income attributable to non-controlling,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,interests in subsidiaries (NCI) $ 22,– $,NaN,22 $ 21,$ 2 $,19 $ 56,$ 4,52
9,Net income attributable to equity holders of t...,13 $,NaN,733 $ 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"


In [32]:
import pandas as pd
import re

# Assuming df8 is already defined with the column 'Unnamed: 0'
# Define a function to split on the first occurrence of a number
def split_on_number(text):
    match = re.search(r'\d', text)
    if match:
        index = match.start()
        return text[:index], text[index:]
    return text, ''

# Apply the function and split into two columns
df8[['(Taxable equivalent basis)', 'Reported ']] = df8['Unnamed: 0'].apply(lambda x: pd.Series(split_on_number(x)))

# Drop the original 'Unnamed: 0' column
df8 = df8.drop(columns=['Unnamed: 0'])

# Reorder columns to place 'Name' and 'Year' as the first and second columns
df8 = df8[['(Taxable equivalent basis)', 'Reported '] + [col for col in df8.columns if col not in ['(Taxable equivalent basis)', 'Reported ']]]

print(df8)


                           (Taxable equivalent basis) Reported  Unnamed: 2  \
0                              Net interest income $      2,246       19 $   
1                                Non-interest income        857          6   
2                                      Total revenue      3,103         25   
3                        Provision for credit losses        574          6   
4                              Non-interest expenses      1,571          2   
5                                 Income tax expense        190          4   
6                                       Net income $        768       13 $   
7          Net income attributable to non-controlling                  NaN   
8                  interests in subsidiaries (NCI) $         22        – $   
9   Net income attributable to equity holders of t...       746       13 $   
10                                     Other measures                  NaN   
11                     Average assets ($ billions) $        236 

In [33]:
df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Unnamed: 4,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,NaN,"2,227 $ 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,NaN,851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078","2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,NaN,568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,NaN,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,NaN,186 171,(10),181 339,(20),359
6,Net income $,768,13 $,NaN,755 $ 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,Net income attributable to non-controlling,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,interests in subsidiaries (NCI) $,22,– $,NaN,22 $ 21,$ 2 $,19 $ 56,$ 4,52
9,Net income attributable to equity holders of t...,746,13 $,NaN,733 $ 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"


In [34]:
# Concatenate 'Unnamed: 3' (4th column) and 'Unnamed: 4' (5th column) into 'Unnamed: 3'
df8['Unnamed: 3'] = df8['Unnamed: 3'].astype(str) + ' ' + df8['Unnamed: 4'].astype(str)

# Drop the 5th column ('Unnamed: 4')
df8 = df8.drop(columns=['Unnamed: 4'])

df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,"nan 2,227 $ 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,nan 851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,nan 568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,"nan 1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,nan 186 171,(10),181 339,(20),359
6,Net income $,768,13 $,nan 755 $ 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,Net income attributable to non-controlling,,NaN,nan nan,NaN,NaN,NaN,NaN
8,interests in subsidiaries (NCI) $,22,– $,nan 22 $ 21,$ 2 $,19 $ 56,$ 4,52
9,Net income attributable to equity holders of t...,746,13 $,nan 733 $ 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"


In [35]:
import pandas as pd

df8['Unnamed: 3'] = df8['Unnamed: 3'].astype(str).str.replace('nan', '', regex=False)
df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,"2,227 $ 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,186 171,(10),181 339,(20),359
6,Net income $,768,13 $,755 $ 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,Net income attributable to non-controlling,,NaN,,NaN,NaN,NaN,NaN
8,interests in subsidiaries (NCI) $,22,– $,22 $ 21,$ 2 $,19 $ 56,$ 4,52
9,Net income attributable to equity holders of t...,746,13 $,733 $ 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"


In [36]:
# Remove dollar signs from the 'Unnamed: 3' column
df8['Unnamed: 3'] = df8['Unnamed: 3'].str.replace('$', '', regex=False)

In [37]:
df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,"2,227 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,186 171,(10),181 339,(20),359
6,Net income $,768,13 $,755 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,Net income attributable to non-controlling,,NaN,,NaN,NaN,NaN,NaN
8,interests in subsidiaries (NCI) $,22,– $,22 21,$ 2 $,19 $ 56,$ 4,52
9,Net income attributable to equity holders of t...,746,13 $,733 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"


In [38]:
# Drop rows with indices from 1 to 7 (inclusive)
df8 = df8.drop(df8.index[7])
df8 = df8.reset_index(drop=True)
df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,"2,227 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,186 171,(10),181 339,(20),359
6,Net income $,768,13 $,755 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,interests in subsidiaries (NCI) $,22,– $,22 21,$ 2 $,19 $ 56,$ 4,52
8,Net income attributable to equity holders of t...,746,13 $,733 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"
9,Other measures,,NaN,,NaN,NaN,NaN,NaN


In [39]:
# Drop rows with indices from 1 to 7 (inclusive)
df8 = df8.drop(df8.index[9])
df8 = df8.reset_index(drop=True)
df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11
0,Net interest income $,"2,246",19 $,"2,227 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973"
1,Non-interest income,857,6,851 743,(88),"831 1,535",(163),"1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671"
3,Provision for credit losses,574,6,568 436,(3),439 840,(27),867
4,Non-interest expenses,"1,571",2,"1,569 1,478","(23) 1,501","2,911",(98),"3,009"
5,Income tax expense,190,4,186 171,(10),181 339,(20),359
6,Net income $,768,13 $,755 657,$ (44) $,"701 $ 1,336",$ (100),"1,436"
7,interests in subsidiaries (NCI) $,22,– $,22 21,$ 2 $,19 $ 56,$ 4,52
8,Net income attributable to equity holders of t...,746,13 $,733 636,$ (46) $,"682 $ 1,280",$ (104),"1,384"
9,Average assets ($ billions) $,236,1 $,235 239,$ 3 $,236 $ 233,$ (2),235


In [40]:
df8.columns

Index(['(Taxable equivalent basis)', 'Reported ', 'Unnamed: 2', 'Unnamed: 3',
       'Global', 'Global.1', 'Unnamed: 9', 'Unnamed: 11'],
      dtype='object')

In [41]:
# Convert all relevant columns to string type
cols_to_convert = ['Reported ', 'Unnamed: 2', 'Unnamed: 3','Global', 'Global.1', 'Unnamed: 9', 'Unnamed: 11']
df8[cols_to_convert] = df8[cols_to_convert].astype(str)

# Define the new column by combining the specified columns
df8['Combined_Column'] = df8[cols_to_convert].apply(lambda row: ' '.join(row), axis=1)

df8

,(Taxable equivalent basis),Reported,Unnamed: 2,Unnamed: 3,Global,Global.1,Unnamed: 9,Unnamed: 11,Combined_Column
0,Net interest income $,"2,246",19 $,"2,227 1,999","$ 8 $ 1,991","$ 3,891",$ (82),"3,973","2,246 19 $ 2,227 1,999 $ 8 $ 1,991 $ 3,891 $..."
1,Non-interest income,857,6,851 743,(88),"831 1,535",(163),"1,698","857 6 851 743 (88) 831 1,535 (163) 1,698"
2,Total revenue,"3,103",25,"3,078 2,742","(80) 2,822","5,426",(245),"5,671","3,103 25 3,078 2,742 (80) 2,822 5,426 (245) 5..."
3,Provision for credit losses,574,6,568 436,(3),439 840,(27),867,574 6 568 436 (3) 439 840 (27) 867
4,Non-interest expenses,"1,571",2,"1,569 1,478","(23) 1,501","2,911",(98),"3,009","1,571 2 1,569 1,478 (23) 1,501 2,911 (98) 3,009"
5,Income tax expense,190,4,186 171,(10),181 339,(20),359,190 4 186 171 (10) 181 339 (20) 359
6,Net income $,768,13 $,755 657,$ (44) $,"701 $ 1,336",$ (100),"1,436","768 13 $ 755 657 $ (44) $ 701 $ 1,336 $ (100..."
7,interests in subsidiaries (NCI) $,22,– $,22 21,$ 2 $,19 $ 56,$ 4,52,22 – $ 22 21 $ 2 $ 19 $ 56 $ 4 52
8,Net income attributable to equity holders of t...,746,13 $,733 636,$ (46) $,"682 $ 1,280",$ (104),"1,384","746 13 $ 733 636 $ (46) $ 682 $ 1,280 $ (104..."
9,Average assets ($ billions) $,236,1 $,235 239,$ 3 $,236 $ 233,$ (2),235,236 1 $ 235 239 $ 3 $ 236 $ 233 $ (2) 235


In [42]:
df8 = df8.drop(columns=['Reported ', 'Unnamed: 2', 'Unnamed: 3', 'Global', 'Global.1', 'Unnamed: 9', 'Unnamed: 11'], errors='ignore')
df8

,(Taxable equivalent basis),Combined_Column
0,Net interest income $,"2,246 19 $ 2,227 1,999 $ 8 $ 1,991 $ 3,891 $..."
1,Non-interest income,"857 6 851 743 (88) 831 1,535 (163) 1,698"
2,Total revenue,"3,103 25 3,078 2,742 (80) 2,822 5,426 (245) 5..."
3,Provision for credit losses,574 6 568 436 (3) 439 840 (27) 867
4,Non-interest expenses,"1,571 2 1,569 1,478 (23) 1,501 2,911 (98) 3,009"
5,Income tax expense,190 4 186 171 (10) 181 339 (20) 359
6,Net income $,"768 13 $ 755 657 $ (44) $ 701 $ 1,336 $ (100..."
7,interests in subsidiaries (NCI) $,22 – $ 22 21 $ 2 $ 19 $ 56 $ 4 52
8,Net income attributable to equity holders of t...,"746 13 $ 733 636 $ (46) $ 682 $ 1,280 $ (104..."
9,Average assets ($ billions) $,236 1 $ 235 239 $ 3 $ 236 $ 233 $ (2) 235


In [43]:
# Remove the $ sign from the Combined_Column
df8['Combined_Column'] = df8['Combined_Column'].str.replace('$', '', regex=False)
df8

,(Taxable equivalent basis),Combined_Column
0,Net interest income $,"2,246 19 2,227 1,999 8 1,991 3,891 (82)..."
1,Non-interest income,"857 6 851 743 (88) 831 1,535 (163) 1,698"
2,Total revenue,"3,103 25 3,078 2,742 (80) 2,822 5,426 (245) 5..."
3,Provision for credit losses,574 6 568 436 (3) 439 840 (27) 867
4,Non-interest expenses,"1,571 2 1,569 1,478 (23) 1,501 2,911 (98) 3,009"
5,Income tax expense,190 4 186 171 (10) 181 339 (20) 359
6,Net income $,"768 13 755 657 (44) 701 1,336 (100) 1,436"
7,interests in subsidiaries (NCI) $,22 – 22 21 2 19 56 4 52
8,Net income attributable to equity holders of t...,"746 13 733 636 (46) 682 1,280 (104) 1,384"
9,Average assets ($ billions) $,236 1 235 239 3 236 233 (2) 235


In [44]:
# Remove the $ sign and brackets from the Combined_Column
df8['Combined_Column'] = df8['Combined_Column'].str.replace(r'[()]', '', regex=True)
df8

,(Taxable equivalent basis),Combined_Column
0,Net interest income $,"2,246 19 2,227 1,999 8 1,991 3,891 82 3..."
1,Non-interest income,"857 6 851 743 88 831 1,535 163 1,698"
2,Total revenue,"3,103 25 3,078 2,742 80 2,822 5,426 245 5,671"
3,Provision for credit losses,574 6 568 436 3 439 840 27 867
4,Non-interest expenses,"1,571 2 1,569 1,478 23 1,501 2,911 98 3,009"
5,Income tax expense,190 4 186 171 10 181 339 20 359
6,Net income $,"768 13 755 657 44 701 1,336 100 1,436"
7,interests in subsidiaries (NCI) $,22 – 22 21 2 19 56 4 52
8,Net income attributable to equity holders of t...,"746 13 733 636 46 682 1,280 104 1,384"
9,Average assets ($ billions) $,236 1 235 239 3 236 233 2 235


In [45]:
# Define the function to split text and values
def split_text_and_values(text):
    # Extract text part (everything before the first number or special character)
    text_match = re.match(r'(.*?)(?=\d|\–)', text)
    text_part = text_match.group(1).strip() if text_match else ''

    # Extract all numeric values with up to 5 digits and special characters like ‘–’
    values = re.findall(r'\d{1,5}(?:,\d{3})*|\–', text)

    # Clean values
    clean_values = [v.strip() for v in values]

    # Ensure that special characters are preserved
    clean_values = [v if v != '–' else '-' for v in clean_values]

    # Ensure the length of the result matches the expected number of columns (9 total)
    num_values = len(clean_values)
    return [text_part] + clean_values + [None] * (9 - num_values)

# Apply the function and expand the results into separate columns
split_data = df8['Combined_Column'].apply(split_text_and_values)
split_df = pd.DataFrame(split_data.tolist(), columns=[f'value_{i+1}' for i in range(10)], index=df8.index)

# Combine with the original DataFrame, dropping the old column
df8_combined = pd.concat([df8.drop(columns=['Combined_Column']), split_df], axis=1)

print(df8_combined)

                           (Taxable equivalent basis) value_1 value_2 value_3  \
0                              Net interest income $            2,246      19   
1                                Non-interest income              857       6   
2                                      Total revenue            3,103      25   
3                        Provision for credit losses              574       6   
4                              Non-interest expenses            1,571       2   
5                                 Income tax expense              190       4   
6                                       Net income $              768      13   
7                  interests in subsidiaries (NCI) $               22       -   
8   Net income attributable to equity holders of t...             746      13   
9                      Average assets ($ billions) $              236       1   
10                Average liabilities ($ billions) $              184       2   

   value_4 value_5 value_6 

In [46]:
df8 = df8_combined
df8

,(Taxable equivalent basis),value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10
0,Net interest income $,,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,,190,4,186,171,10,181,339,20,359
6,Net income $,,768,13,755,657,44,701,"1,336",100,"1,436"
7,interests in subsidiaries (NCI) $,,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ($ billions) $,,236,1,235,239,3,236,233,2,235


In [47]:
df8 = df8.drop(columns='value_1')
df8

,(Taxable equivalent basis),value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10
0,Net interest income $,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,190,4,186,171,10,181,339,20,359
6,Net income $,768,13,755,657,44,701,"1,336",100,"1,436"
7,interests in subsidiaries (NCI) $,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ($ billions) $,236,1,235,239,3,236,233,2,235


In [48]:
# Remove the $ sign from the (Taxable equivalent basis)
df8['(Taxable equivalent basis)'] = df8['(Taxable equivalent basis)'].str.replace('$', '', regex=False)
df8

,(Taxable equivalent basis),value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,190,4,186,171,10,181,339,20,359
6,Net income,768,13,755,657,44,701,"1,336",100,"1,436"
7,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ( billions),236,1,235,239,3,236,233,2,235


In [49]:
df8.rename(columns={
    '(Taxable equivalent basis)': 'Reported Results ($ millions) (Taxable equivalent basis)',
    'value_2': 'Reported (January 31, 2024⁽¹⁾)(For the three months ended)',
    'value_3': 'Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)',
    'value_4': 'Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)',
    'value_5': 'Reported (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_6': 'Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_7': 'Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_8': 'Reported (April 30, 2023⁽¹⁾)(For the six months ended)',
    'value_9': 'Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)',
    'value_10': 'Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)'
}, inplace=True)
df8

,Reported Results ($ millions) (Taxable equivalent basis),"Reported (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,190,4,186,171,10,181,339,20,359
6,Net income,768,13,755,657,44,701,"1,336",100,"1,436"
7,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ( billions),236,1,235,239,3,236,233,2,235


In [50]:
# Renaming the cell value at index 7
df8.at[7, 'Reported Results ($ millions) (Taxable equivalent basis)'] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df8

,Reported Results ($ millions) (Taxable equivalent basis),"Reported (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,190,4,186,171,10,181,339,20,359
6,Net income,768,13,755,657,44,701,"1,336",100,"1,436"
7,Net income attributable to non-controlling int...,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ( billions),236,1,235,239,3,236,233,2,235


#### Cleaned Table 8

In [51]:
df8

,Reported Results ($ millions) (Taxable equivalent basis),"Reported (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Reported (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,571",2,"1,569","1,478",23,"1,501","2,911",98,"3,009"
5,Income tax expense,190,4,186,171,10,181,339,20,359
6,Net income,768,13,755,657,44,701,"1,336",100,"1,436"
7,Net income attributable to non-controlling int...,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,746,13,733,636,46,682,"1,280",104,"1,384"
9,Average assets ( billions),236,1,235,239,3,236,233,2,235


### Table 9

In [52]:
# Extract tables from the PDF on page 11
tables = tabula.read_pdf(pdf_path, pages=11, multiple_tables=True)

# Convert the first table in the list to a DataFrame and limit to the first 27 columns
df9 = pd.DataFrame(tables[0])
df9 = df9.iloc[:13]
df9


Aug 03, 2024 5:34:59 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



,"($ millions) January 31, 2024(1)","April 30, 2023(1)","April 30, 2023(1).1"
0,Constant,Constant,Constant
1,Foreign dollar,Foreign dollar,Foreign dollar
2,(Taxable equivalent basis) Adjusted exchange a...,Adjusted exchange adjusted,Adjusted exchange adjusted
3,"Net interest income $ 2,246 $ 19 $ 2,227","$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973"
4,Non-interest income 857 6 851,743 (88) 831,"1,535 (163) 1,698"
5,"Total revenue 3,103 25 3,078","2,742 (80) 2,822","5,426 (245) 5,671"
6,Provision for credit losses 574 6 568,436 (3) 439,840 (27) 867
7,"Non-interest expenses 1,563 2 1,561","1,467 (24) 1,491","2,890 (99) 2,989"
8,Income tax expense 192 4 188,174 (10) 184,345 (20) 365
9,Net income $ 774 $ 13 $ 761,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450"


In [53]:
import pandas as pd
import re

def split_text_and_values(text):
    # Extract text part (everything before the first number or special character)
    text_match = re.match(r'(.*?)(?=\$|\d)', text)
    text_part = text_match.group(1).strip() if text_match else ''

    # Extract all numeric values and special characters like ‘–’
    values = re.findall(r'(\$?\s?\d{1,3}(?:,\d{3})*|\–)', text)

    # Clean values: remove dollar signs and extra spaces
    clean_values = [v.replace('$', '').strip() for v in values]

    # Ensure that special characters are preserved
    clean_values = [v if v != '–' else '-' for v in clean_values]

    # Ensure the length of the result matches expected columns
    num_values = len(clean_values)
    return [text_part] + clean_values + [None] * (3 - num_values)

# Apply the function and expand the results into separate columns
split_data = df9['($ millions) January 31, 2024(1)'].apply(split_text_and_values)
df9[['text_column', 'value_1', 'value_2', 'value_3']] = pd.DataFrame(split_data.tolist(), index=df9.index)

# Optionally, drop the original column
df9 = df9.drop(columns='($ millions) January 31, 2024(1)')
df9

,"April 30, 2023(1)","April 30, 2023(1).1",text_column,value_1,value_2,value_3
0,Constant,Constant,,None,None,None
1,Foreign dollar,Foreign dollar,,None,None,None
2,Adjusted exchange adjusted,Adjusted exchange adjusted,,None,None,None
3,"$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973",Net interest income,"2,246",19,"2,227"
4,743 (88) 831,"1,535 (163) 1,698",Non-interest income,857,6,851
5,"2,742 (80) 2,822","5,426 (245) 5,671",Total revenue,"3,103",25,"3,078"
6,436 (3) 439,840 (27) 867,Provision for credit losses,574,6,568
7,"1,467 (24) 1,491","2,890 (99) 2,989",Non-interest expenses,"1,563",2,"1,561"
8,174 (10) 184,345 (20) 365,Income tax expense,192,4,188
9,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450",Net income,774,13,761


In [54]:
# Drop the first 3 rows
df9 = df9.drop(df9.index[:3])
df9

,"April 30, 2023(1)","April 30, 2023(1).1",text_column,value_1,value_2,value_3
3,"$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973",Net interest income,"2,246",19,"2,227"
4,743 (88) 831,"1,535 (163) 1,698",Non-interest income,857,6,851
5,"2,742 (80) 2,822","5,426 (245) 5,671",Total revenue,"3,103",25,"3,078"
6,436 (3) 439,840 (27) 867,Provision for credit losses,574,6,568
7,"1,467 (24) 1,491","2,890 (99) 2,989",Non-interest expenses,"1,563",2,"1,561"
8,174 (10) 184,345 (20) 365,Income tax expense,192,4,188
9,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450",Net income,774,13,761
10,NaN,NaN,,None,None,None
11,$ 21 $ 2 $ 19,$ 56 $ 4 $ 52,interests in subsidiaries (NCI),22,-,22
12,$ 644 $ (45) $ 689,"$ 1,295 $ (103) $ 1,398",Net income attributable to equity holders of t...,752,13,739


In [55]:
df9 = df9.reset_index(drop=True)
df9

,"April 30, 2023(1)","April 30, 2023(1).1",text_column,value_1,value_2,value_3
0,"$ 1,999 $ 8 $ 1,991","$ 3,891 $ (82) $ 3,973",Net interest income,"2,246",19,"2,227"
1,743 (88) 831,"1,535 (163) 1,698",Non-interest income,857,6,851
2,"2,742 (80) 2,822","5,426 (245) 5,671",Total revenue,"3,103",25,"3,078"
3,436 (3) 439,840 (27) 867,Provision for credit losses,574,6,568
4,"1,467 (24) 1,491","2,890 (99) 2,989",Non-interest expenses,"1,563",2,"1,561"
5,174 (10) 184,345 (20) 365,Income tax expense,192,4,188
6,$ 665 $ (43) $ 708,"$ 1,351 $ (99) $ 1,450",Net income,774,13,761
7,NaN,NaN,,None,None,None
8,$ 21 $ 2 $ 19,$ 56 $ 4 $ 52,interests in subsidiaries (NCI),22,-,22
9,$ 644 $ (45) $ 689,"$ 1,295 $ (103) $ 1,398",Net income attributable to equity holders of t...,752,13,739


In [56]:
import pandas as pd
import re

# Define a function to split numeric values
def split_numeric_values(text):
    if isinstance(text, str):  # Ensure text is a string
        # Extract all numeric values, including those within brackets
        values = re.findall(r'\$?\s*\(?(\d{1,3}(?:,\d{3})*)\)?', text)
        # Return the extracted values, padding with None if fewer than 3 values
        return values + [None] * (3 - len(values))
    else:
        # If not a string, return None values for each expected column
        return [None, None, None]

# Apply the function and expand the results into separate columns
split_data = df9['April 30, 2023(1)'].apply(split_numeric_values)
df9[['value_4', 'value_5', 'value_6']] = pd.DataFrame(split_data.tolist(), index=df9.index)

# Optionally, drop the original column
df9 = df9.drop(columns='April 30, 2023(1)')
df9

,"April 30, 2023(1).1",text_column,value_1,value_2,value_3,value_4,value_5,value_6
0,"$ 3,891 $ (82) $ 3,973",Net interest income,"2,246",19,"2,227","1,999",8,"1,991"
1,"1,535 (163) 1,698",Non-interest income,857,6,851,743,88,831
2,"5,426 (245) 5,671",Total revenue,"3,103",25,"3,078","2,742",80,"2,822"
3,840 (27) 867,Provision for credit losses,574,6,568,436,3,439
4,"2,890 (99) 2,989",Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491"
5,345 (20) 365,Income tax expense,192,4,188,174,10,184
6,"$ 1,351 $ (99) $ 1,450",Net income,774,13,761,665,43,708
7,NaN,,None,None,None,None,None,None
8,$ 56 $ 4 $ 52,interests in subsidiaries (NCI),22,-,22,21,2,19
9,"$ 1,295 $ (103) $ 1,398",Net income attributable to equity holders of t...,752,13,739,644,45,689


In [57]:
import pandas as pd
import re

# Define a function to split numeric values
def split_numeric_values(text):
    if isinstance(text, str):  # Ensure text is a string
        # Extract all numeric values, including those within brackets
        values = re.findall(r'\$?\s*\(?(\d{1,3}(?:,\d{3})*)\)?', text)
        # Return the extracted values, padding with None if fewer than 3 values
        return values + [None] * (3 - len(values))
    else:
        # If not a string, return None values for each expected column
        return [None, None, None]

# Apply the function and expand the results into separate columns
split_data = df9['April 30, 2023(1).1'].apply(split_numeric_values)
df9[['value_7', 'value_8', 'value_9']] = pd.DataFrame(split_data.tolist(), index=df9.index)

# Optionally, drop the original column
df9 = df9.drop(columns='April 30, 2023(1).1')
df9

,text_column,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,,None,None,None,None,None,None,None,None,None
8,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
9,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


In [58]:
# Drop the 8th row (index 7)
df9 = df9.drop(df9.index[7])

# Reset the index after dropping the row
df9 = df9.reset_index(drop=True)
df9

,text_column,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


In [59]:
df9.rename(columns={
    'text_column': 'Adjusted Results ($ millions) (Taxable equivalent basis) ',
    'value_1': 'Adjusted (January 31, 2024⁽¹⁾)(For the three months ended) ',
    'value_2': 'Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)',
    'value_3': 'Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)',
    'value_4': 'Adjusted (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_5': 'Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_6': 'Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)',
    'value_7': 'Adjusted (April 30, 2023⁽¹⁾)(For the six months ended)',
    'value_8': 'Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)',
    'value_9': 'Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)'
}, inplace=True)
df9

,Adjusted Results ($ millions) (Taxable equivalent basis),"Adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


In [60]:
df9

,Adjusted Results ($ millions) (Taxable equivalent basis),"Adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,interests in subsidiaries (NCI),22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


In [61]:
# Renaming the cell value at index 7
df9.at[7, 'Adjusted Results ($ millions) (Taxable equivalent basis) '] = 'Net income attributable to non-controlling interests in subsidiaries (NCI)'
df9

,Adjusted Results ($ millions) (Taxable equivalent basis),"Adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,Net income attributable to non-controlling int...,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


#### Cleaned Table 9

In [62]:
df9

,Adjusted Results ($ millions) (Taxable equivalent basis),"Adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Foreign exchange (January 31, 2024⁽¹⁾)(For the three months ended)","Constant dollar adjusted (January 31, 2024⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the three months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the three months ended)","Adjusted (April 30, 2023⁽¹⁾)(For the six months ended)","Foreign exchange (April 30, 2023⁽¹⁾)(For the six months ended)","Constant dollar adjusted (April 30, 2023⁽¹⁾)(For the six months ended)"
0,Net interest income,"2,246",19,"2,227","1,999",8,"1,991","3,891",82,"3,973"
1,Non-interest income,857,6,851,743,88,831,"1,535",163,"1,698"
2,Total revenue,"3,103",25,"3,078","2,742",80,"2,822","5,426",245,"5,671"
3,Provision for credit losses,574,6,568,436,3,439,840,27,867
4,Non-interest expenses,"1,563",2,"1,561","1,467",24,"1,491","2,890",99,"2,989"
5,Income tax expense,192,4,188,174,10,184,345,20,365
6,Net income,774,13,761,665,43,708,"1,351",99,"1,450"
7,Net income attributable to non-controlling int...,22,-,22,21,2,19,56,4,52
8,Net income attributable to equity holders of t...,752,13,739,644,45,689,"1,295",103,"1,398"


### Table 10

In [63]:
# Extract tables from the PDF on page 11
tables = tabula.read_pdf(pdf_path, pages=11, multiple_tables=True)

# Convert the first table in the list to a DataFrame and limit to the first 12 columns
df10 = pd.DataFrame(tables[0])
# Select all rows starting from the 35th row and reset the index
df10 = df10.iloc[35:].reset_index(drop=True)

# Set the 210th row (now the first row in df6) as the header
df10.columns = df10.iloc[0]
df10 = df10[1:]
df10

Aug 03, 2024 5:35:05 PM org.apache.fontbox.ttf.CmapSubtable processSubtype14



,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
1,Global Global,Global,Global
2,Canadian International Wealth Banking and,Canadian International Wealth,Banking and
3,($ millions) Banking(1) Banking(1) Management ...,Banking(1) Banking(1) Management,Markets Other Total(1)
4,Reported,NaN,NaN
5,Net income,NaN,NaN
6,attributable,NaN,NaN
7,to common,NaN,NaN
8,"shareholders $ 1,008 $ 671 $ 380 $ 428 $ (544...","$ 1,054 $ 635 $","352 $ 400 $ (423) $ 2,018"
9,Total average,NaN,NaN
10,common,NaN,NaN


In [64]:
# Drop rows with indices from 1 to 7 (inclusive)
df10 = df10.drop(df10.index[1:7])

# Reset the index to keep it continuous after dropping rows (optional)
df10 = df10.reset_index(drop=True)
df10

,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,Global Global,Global,Global
1,"shareholders $ 1,008 $ 671 $ 380 $ 428 $ (544...","$ 1,054 $ 635 $","352 $ 400 $ (423) $ 2,018"
2,Total average,NaN,NaN
3,common,NaN,NaN
4,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
5,Return on equity 20.0% 14.4% 15.1% 11.7% nm(4)...,22.7% 13.1% 14.8%,10.5% nm(4) 12.2%
6,Adjusted(5),NaN,NaN
7,Net income,NaN,NaN
8,attributable,NaN,NaN
9,to common,NaN,NaN


In [65]:
# Drop rows with indices from 1 to 7 (inclusive)
df10 = df10.drop(df10.index[2:4])

# Reset the index to keep it continuous after dropping rows (optional)
df10 = df10.reset_index(drop=True)
df10

,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,Global Global,Global,Global
1,"shareholders $ 1,008 $ 671 $ 380 $ 428 $ (544...","$ 1,054 $ 635 $","352 $ 400 $ (423) $ 2,018"
2,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
3,Return on equity 20.0% 14.4% 15.1% 11.7% nm(4)...,22.7% 13.1% 14.8%,10.5% nm(4) 12.2%
4,Adjusted(5),NaN,NaN
5,Net income,NaN,NaN
6,attributable,NaN,NaN
7,to common,NaN,NaN
8,"shareholders $ 1,008 $ 677 $ 387 $ 428 $ (544...","$ 1,055 $ 643 $","358 $ 400 $ (423) $ 2,033"
9,Return on equity 20.0% 14.5% 15.4% 11.7% nm(4)...,22.7% 13.3%,15.1% 10.5% nm(4) 12.3%


In [66]:
# Drop rows with indices from 1 to 7 (inclusive)
df10 = df10.drop(df10.index[4:8])

# Reset the index to keep it continuous after dropping rows (optional)
df10 = df10.reset_index(drop=True)
df10

,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,Global Global,Global,Global
1,"shareholders $ 1,008 $ 671 $ 380 $ 428 $ (544...","$ 1,054 $ 635 $","352 $ 400 $ (423) $ 2,018"
2,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
3,Return on equity 20.0% 14.4% 15.1% 11.7% nm(4)...,22.7% 13.1% 14.8%,10.5% nm(4) 12.2%
4,"shareholders $ 1,008 $ 677 $ 387 $ 428 $ (544...","$ 1,055 $ 643 $","358 $ 400 $ (423) $ 2,033"
5,Return on equity 20.0% 14.5% 15.4% 11.7% nm(4)...,22.7% 13.3%,15.1% 10.5% nm(4) 12.3%


In [67]:
# Drop rows with indices from 1 to 7 (inclusive)
df10 = df10.drop(df10.index[0])

# Reset the index to keep it continuous after dropping rows (optional)
df10 = df10.reset_index(drop=True)
df10

,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,"shareholders $ 1,008 $ 671 $ 380 $ 428 $ (544...","$ 1,054 $ 635 $","352 $ 400 $ (423) $ 2,018"
1,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
2,Return on equity 20.0% 14.4% 15.1% 11.7% nm(4)...,22.7% 13.1% 14.8%,10.5% nm(4) 12.2%
3,"shareholders $ 1,008 $ 677 $ 387 $ 428 $ (544...","$ 1,055 $ 643 $","358 $ 400 $ (423) $ 2,033"
4,Return on equity 20.0% 14.5% 15.4% 11.7% nm(4)...,22.7% 13.3%,15.1% 10.5% nm(4) 12.3%


In [68]:
# Remove $ and % from all cells
df10 = df10.applymap(lambda x: x.replace('$', '').replace('%', '') if isinstance(x, str) else x)
df10

<ipython-input-68-66ad076c4236>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df10 = df10.applymap(lambda x: x.replace('$', '').replace('%', '') if isinstance(x, str) else x)


,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,"shareholders 1,008 671 380 428 (544) 1,943","1,054 635","352 400 (423) 2,018"
1,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
2,Return on equity 20.0 14.4 15.1 11.7 nm(4) 11.2,22.7 13.1 14.8,10.5 nm(4) 12.2
3,"shareholders 1,008 677 387 428 (544) 1,956","1,055 643","358 400 (423) 2,033"
4,Return on equity 20.0 14.5 15.4 11.7 nm(4) 11.3,22.7 13.3,15.1 10.5 nm(4) 12.3


In [69]:
df10

,"For the three months ended April 30, 2024","For the three months ended April 30, 2023",NaN
0,"shareholders 1,008 671 380 428 (544) 1,943","1,054 635","352 400 (423) 2,018"
1,"equity(2)(3) 20,507 18,927 10,222 14,865 5,756...","19,077 19,866","9,732 15,587 3,312 67,574"
2,Return on equity 20.0 14.4 15.1 11.7 nm(4) 11.2,22.7 13.1 14.8,10.5 nm(4) 12.2
3,"shareholders 1,008 677 387 428 (544) 1,956","1,055 643","358 400 (423) 2,033"
4,Return on equity 20.0 14.5 15.4 11.7 nm(4) 11.3,22.7 13.3,15.1 10.5 nm(4) 12.3


In [70]:
import re
import pandas as pd

def split_text_digits(text):
    # Extract text part, including parentheses attached to text
    text_part = ' '.join(re.findall(r'[a-zA-Z\s]+(?:\([a-zA-Z]+\))?', text)).strip()
    # Extract numeric part, including numbers with commas and parentheses around numbers
    digits_part = ' '.join(re.findall(r'[0-9,]+(?:\.[0-9]+)?(?:\([0-9]+\))?', text)).strip()
    return pd.Series([text_part, digits_part])

# Apply the function and create new columns
df10[['text_column', 'digits_column']] = df10['For the three months ended April 30, 2024'].apply(split_text_digits)

# Drop the old column
df10 = df10.drop(columns=['For the three months ended April 30, 2024'])

# Display the updated DataFrame
df10


,"For the three months ended April 30, 2023",NaN,text_column,digits_column
0,"1,054 635","352 400 (423) 2,018",shareholders,"1,008 671 380 428 544 1,943"
1,"19,077 19,866","9,732 15,587 3,312 67,574",equity,"2 3 20,507 18,927 10,222 14,865 5,756 70,277"
2,22.7 13.1 14.8,10.5 nm(4) 12.2,Return on equity nm,20.0 14.4 15.1 11.7 4 11.2
3,"1,055 643","358 400 (423) 2,033",shareholders,"1,008 677 387 428 544 1,956"
4,22.7 13.3,15.1 10.5 nm(4) 12.3,Return on equity nm,20.0 14.5 15.4 11.7 4 11.3


In [71]:
# Define the function to remove specific values from a text
def remove_specific_values(text, values_to_remove):
    # Convert the text into a list of items
    items = text.split()

    # Remove specified values
    filtered_items = [item for item in items if item not in values_to_remove]

    # Join the list back into a string
    return ' '.join(filtered_items)

# Values to remove
values_to_remove = {'2', '3'}

# Apply the function to the specific row and column
row_index = 1  # Index of the row you want to modify (e.g., 0 for the first row)
df10.at[row_index, 'digits_column'] = remove_specific_values(df10.at[row_index, 'digits_column'], values_to_remove)
df10

,"For the three months ended April 30, 2023",NaN,text_column,digits_column
0,"1,054 635","352 400 (423) 2,018",shareholders,"1,008 671 380 428 544 1,943"
1,"19,077 19,866","9,732 15,587 3,312 67,574",equity,"20,507 18,927 10,222 14,865 5,756 70,277"
2,22.7 13.1 14.8,10.5 nm(4) 12.2,Return on equity nm,20.0 14.4 15.1 11.7 4 11.2
3,"1,055 643","358 400 (423) 2,033",shareholders,"1,008 677 387 428 544 1,956"
4,22.7 13.3,15.1 10.5 nm(4) 12.3,Return on equity nm,20.0 14.5 15.4 11.7 4 11.3


In [72]:
df10.columns.values[1] = 'Col'
df10

,"For the three months ended April 30, 2023",Col,text_column,digits_column
0,"1,054 635","352 400 (423) 2,018",shareholders,"1,008 671 380 428 544 1,943"
1,"19,077 19,866","9,732 15,587 3,312 67,574",equity,"20,507 18,927 10,222 14,865 5,756 70,277"
2,22.7 13.1 14.8,10.5 nm(4) 12.2,Return on equity nm,20.0 14.4 15.1 11.7 4 11.2
3,"1,055 643","358 400 (423) 2,033",shareholders,"1,008 677 387 428 544 1,956"
4,22.7 13.3,15.1 10.5 nm(4) 12.3,Return on equity nm,20.0 14.5 15.4 11.7 4 11.3


In [73]:
import pandas as pd
import re

# Define a function to split numeric values
def split_numeric_values(text):
    if isinstance(text, str):  # Ensure text is a string
        # Extract all numeric values, including those within brackets
        values = re.findall(r'\$?\s*\(?(\d+(?:,\d{3})*(?:\.\d+)?)\)?', text)
        # Convert values to floats, if possible
        values = [float(val.replace(',', '')) if val else None for val in values]
        # Return the extracted values, padding with None if fewer than 6 values
        return values + [None] * (6 - len(values))
    else:
        # If not a string, return None values for each expected column
        return [None] * 6

# Apply the function and expand the results into separate columns
split_data = df10['digits_column'].apply(split_numeric_values)
split_df = pd.DataFrame(split_data.tolist(), index=df10.index)

# Assign the split columns to the original DataFrame
df10[['value_1', 'value_2', 'value_3', 'value_4', 'value_5', 'value_6']] = split_df

# Optionally, drop the original column
df10 = df10.drop(columns='digits_column')
df10


,"For the three months ended April 30, 2023",Col,text_column,value_1,value_2,value_3,value_4,value_5,value_6
0,"1,054 635","352 400 (423) 2,018",shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0
1,"19,077 19,866","9,732 15,587 3,312 67,574",equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0
2,22.7 13.1 14.8,10.5 nm(4) 12.2,Return on equity nm,20.0,14.4,15.1,11.7,4.0,11.2
3,"1,055 643","358 400 (423) 2,033",shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0
4,22.7 13.3,15.1 10.5 nm(4) 12.3,Return on equity nm,20.0,14.5,15.4,11.7,4.0,11.3


In [74]:


# Convert both columns to string type to ensure concatenation works
df10['Col'] = df10['Col'].astype(str)
df10['For the three months ended April 30, 2023'] = df10['For the three months ended April 30, 2023'].astype(str)

# Define the new column by combining 'Col' and 'For the three months ended April 30, 2023'
df10['Combined_Column'] =  df10['For the three months ended April 30, 2023']+ ' ' +df10['Col']


# Display the updated DataFrame
df10

,"For the three months ended April 30, 2023",Col,text_column,value_1,value_2,value_3,value_4,value_5,value_6,Combined_Column
0,"1,054 635","352 400 (423) 2,018",shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0,"1,054 635 352 400 (423) 2,018"
1,"19,077 19,866","9,732 15,587 3,312 67,574",equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,"19,077 19,866 9,732 15,587 3,312 67,574"
2,22.7 13.1 14.8,10.5 nm(4) 12.2,Return on equity nm,20.0,14.4,15.1,11.7,4.0,11.2,22.7 13.1 14.8 10.5 nm(4) 12.2
3,"1,055 643","358 400 (423) 2,033",shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0,"1,055 643 358 400 (423) 2,033"
4,22.7 13.3,15.1 10.5 nm(4) 12.3,Return on equity nm,20.0,14.5,15.4,11.7,4.0,11.3,22.7 13.3 15.1 10.5 nm(4) 12.3


In [75]:
# Optionally, drop the original columns if no longer needed
df10 = df10.drop(columns=['Col', 'For the three months ended April 30, 2023'])
df10


,text_column,value_1,value_2,value_3,value_4,value_5,value_6,Combined_Column
0,shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0,"1,054 635 352 400 (423) 2,018"
1,equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,"19,077 19,866 9,732 15,587 3,312 67,574"
2,Return on equity nm,20.0,14.4,15.1,11.7,4.0,11.2,22.7 13.1 14.8 10.5 nm(4) 12.2
3,shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0,"1,055 643 358 400 (423) 2,033"
4,Return on equity nm,20.0,14.5,15.4,11.7,4.0,11.3,22.7 13.3 15.1 10.5 nm(4) 12.3


In [76]:
import pandas as pd
import re

# Define a function to split numeric values
def split_numeric_values(text):
    if isinstance(text, str):  # Ensure text is a string
        # Extract all numeric values, including those within brackets
        values = re.findall(r'\$?\s*\(?(\d+(?:,\d{3})*(?:\.\d+)?)\)?', text)
        # Convert values to floats, if possible
        values = [float(val.replace(',', '')) if val else None for val in values]
        # Return the extracted values, padding with None if fewer than 6 values
        return values + [None] * (6 - len(values))
    else:
        # If not a string, return None values for each expected column
        return [None] * 6

# Apply the function and expand the results into separate columns
split_data = df10['Combined_Column'].apply(split_numeric_values)
split_df = pd.DataFrame(split_data.tolist(), index=df10.index)

# Assign the split columns to the original DataFrame
df10[['value_7', 'value_8', 'value_9', 'value_10', 'value_11', 'value_12']] = split_df

# Optionally, drop the original column
df10 = df10.drop(columns='Combined_Column')
df10


,text_column,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12
0,shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0,1054.0,635.0,352.0,400.0,423.0,2018.0
1,equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,19077.0,19866.0,9732.0,15587.0,3312.0,67574.0
2,Return on equity nm,20.0,14.4,15.1,11.7,4.0,11.2,22.7,13.1,14.8,10.5,4.0,12.2
3,shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0,1055.0,643.0,358.0,400.0,423.0,2033.0
4,Return on equity nm,20.0,14.5,15.4,11.7,4.0,11.3,22.7,13.3,15.1,10.5,4.0,12.3


In [77]:
# Define a function to replace '4' with 'nm(4)'
def replace_4_with_nm(text):
    # Use regular expression to replace '4' with 'nm(4)'
    return re.sub(r'\b4\b', 'nm(4)', text)

# Apply the function to 'value_5' and 'value_11'
df10['value_5'] = df10['value_5'].apply(lambda x: replace_4_with_nm(str(x)))
df10['value_11'] = df10['value_11'].apply(lambda x: replace_4_with_nm(str(x)))
df10

,text_column,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9,value_10,value_11,value_12
0,shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0,1054.0,635.0,352.0,400.0,423.0,2018.0
1,equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,19077.0,19866.0,9732.0,15587.0,3312.0,67574.0
2,Return on equity nm,20.0,14.4,15.1,11.7,nm(4).0,11.2,22.7,13.1,14.8,10.5,nm(4).0,12.2
3,shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0,1055.0,643.0,358.0,400.0,423.0,2033.0
4,Return on equity nm,20.0,14.5,15.4,11.7,nm(4).0,11.3,22.7,13.3,15.1,10.5,nm(4).0,12.3


In [78]:
df10.rename(columns={
    'text_column': '($ millions)',
    'value_1': 'Canadian Banking(1) (For the three months ended April 30, 2024) ',
    'value_2': 'International Banking(1) (For the three months ended April 30, 2024)',
    'value_3': 'Global Wealth Management (For the three months ended April 30, 2024)',
    'value_4': 'Global Banking and Markets (For the three months ended April 30, 2024)',
    'value_5': 'Other (For the three months ended April 30, 2024)',
    'value_6': 'Total (For the three months ended April 30, 2024)',
    'value_7': 'Canadian Banking(1) (For the three months ended April 30, 2023)',
    'value_8': 'International Banking(1) (For the three months ended April 30, 2023)',
    'value_9': 'Global Wealth Management (For the three months ended April 30, 2023)',
    'value_10': 'Global Banking and Markets (For the three months ended April 30, 2023)',
    'value_11': 'Other (For the three months ended April 30, 2023)',
    'value_12': 'Total (For the three months ended April 30, 2023)'
}, inplace=True)
df10

,($ millions),"Canadian Banking(1) (For the three months ended April 30, 2024)","International Banking(1) (For the three months ended April 30, 2024)","Global Wealth Management (For the three months ended April 30, 2024)","Global Banking and Markets (For the three months ended April 30, 2024)","Other (For the three months ended April 30, 2024)","Total (For the three months ended April 30, 2024)","Canadian Banking(1) (For the three months ended April 30, 2023)","International Banking(1) (For the three months ended April 30, 2023)","Global Wealth Management (For the three months ended April 30, 2023)","Global Banking and Markets (For the three months ended April 30, 2023)","Other (For the three months ended April 30, 2023)","Total (For the three months ended April 30, 2023)"
0,shareholders,1008.0,671.0,380.0,428.0,544.0,1943.0,1054.0,635.0,352.0,400.0,423.0,2018.0
1,equity,20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,19077.0,19866.0,9732.0,15587.0,3312.0,67574.0
2,Return on equity nm,20.0,14.4,15.1,11.7,nm(4).0,11.2,22.7,13.1,14.8,10.5,nm(4).0,12.2
3,shareholders,1008.0,677.0,387.0,428.0,544.0,1956.0,1055.0,643.0,358.0,400.0,423.0,2033.0
4,Return on equity nm,20.0,14.5,15.4,11.7,nm(4).0,11.3,22.7,13.3,15.1,10.5,nm(4).0,12.3


In [79]:
# Renaming the rows name

df10.iloc[0, df10.columns.get_loc('($ millions)')] = 'Reported Net income attributable to common shareholders',
df10.iloc[1, df10.columns.get_loc('($ millions)')] = 'Total average common equity(2)(3)',
df10.iloc[2, df10.columns.get_loc('($ millions)')] = 'Return on equity',
df10.iloc[3, df10.columns.get_loc('($ millions)')] = 'Adjusted(5) Net income attributable to common shareholders',
df10.iloc[4, df10.columns.get_loc('($ millions)')] = 'Return on equity'
df10

,($ millions),"Canadian Banking(1) (For the three months ended April 30, 2024)","International Banking(1) (For the three months ended April 30, 2024)","Global Wealth Management (For the three months ended April 30, 2024)","Global Banking and Markets (For the three months ended April 30, 2024)","Other (For the three months ended April 30, 2024)","Total (For the three months ended April 30, 2024)","Canadian Banking(1) (For the three months ended April 30, 2023)","International Banking(1) (For the three months ended April 30, 2023)","Global Wealth Management (For the three months ended April 30, 2023)","Global Banking and Markets (For the three months ended April 30, 2023)","Other (For the three months ended April 30, 2023)","Total (For the three months ended April 30, 2023)"
0,(Reported Net income attributable to common sh...,1008.0,671.0,380.0,428.0,544.0,1943.0,1054.0,635.0,352.0,400.0,423.0,2018.0
1,"(Total average common equity(2)(3),)",20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,19077.0,19866.0,9732.0,15587.0,3312.0,67574.0
2,"(Return on equity,)",20.0,14.4,15.1,11.7,nm(4).0,11.2,22.7,13.1,14.8,10.5,nm(4).0,12.2
3,(Adjusted(5) Net income attributable to common...,1008.0,677.0,387.0,428.0,544.0,1956.0,1055.0,643.0,358.0,400.0,423.0,2033.0
4,Return on equity,20.0,14.5,15.4,11.7,nm(4).0,11.3,22.7,13.3,15.1,10.5,nm(4).0,12.3


#### Cleaned Table 10

In [80]:
df10

,($ millions),"Canadian Banking(1) (For the three months ended April 30, 2024)","International Banking(1) (For the three months ended April 30, 2024)","Global Wealth Management (For the three months ended April 30, 2024)","Global Banking and Markets (For the three months ended April 30, 2024)","Other (For the three months ended April 30, 2024)","Total (For the three months ended April 30, 2024)","Canadian Banking(1) (For the three months ended April 30, 2023)","International Banking(1) (For the three months ended April 30, 2023)","Global Wealth Management (For the three months ended April 30, 2023)","Global Banking and Markets (For the three months ended April 30, 2023)","Other (For the three months ended April 30, 2023)","Total (For the three months ended April 30, 2023)"
0,(Reported Net income attributable to common sh...,1008.0,671.0,380.0,428.0,544.0,1943.0,1054.0,635.0,352.0,400.0,423.0,2018.0
1,"(Total average common equity(2)(3),)",20507.0,18927.0,10222.0,14865.0,5756.0,70277.0,19077.0,19866.0,9732.0,15587.0,3312.0,67574.0
2,"(Return on equity,)",20.0,14.4,15.1,11.7,nm(4).0,11.2,22.7,13.1,14.8,10.5,nm(4).0,12.2
3,(Adjusted(5) Net income attributable to common...,1008.0,677.0,387.0,428.0,544.0,1956.0,1055.0,643.0,358.0,400.0,423.0,2033.0
4,Return on equity,20.0,14.5,15.4,11.7,nm(4).0,11.3,22.7,13.3,15.1,10.5,nm(4).0,12.3
